In [ ]:
import json
import time
import pandas
import requests
from bs4 import BeautifulSoup

In [ ]:
!pip install trafilatura
import os
import re
import shutil
import trafilatura

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 3.8 MB/s eta 0:00:00


In [ ]:
url = "https://www.stonerealestate.com.au/blog/"
res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")
soup

<!DOCTYPE html>

<html dir="ltr" itemscope="" itemtype="https://schema.org/WebPage" lang="en-US" xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml">
<head profile="https://gmpg.org/xfn/11">
<meta content="width=device-width, minimum-scale=1.0, maximum-scale=1.0, user-scalable=yes" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<meta content="6EC952A5D4EDCC4985A8FB51BAB6A322" name="msvalidate.01"/>
<meta content="fdzhp9ph8c30g0zg90tt6qvpwryace" name="facebook-domain-verification"/>
<meta content="r9xx20w42n8dmyocoxlsx19uzkpb6t" name="facebook-domain-verification"/>
<meta content="Sw5N05NFdh0JqTrFfElpwr8tBjJq7VmogHwBVrWYobg" name="google-site-verification"/>
<title>Stone Real Estate | Blog | Stone Real Estate</title>
<link href="https://www.stonerealestate.com.au/wp-content/uploads/2017/08/fav-stone.png" rel="icon" type="image/png"/>
<link href="https://www.stonerealestate.com.au/feed/" rel="alternate" title="Stone R

In [ ]:
from urllib.parse import urljoin, urlparse

def get_stone_realestate_urls(soup, base_url="https://www.stonerealestate.com.au"):
    urls = set()

    for a in soup.find_all("a", href=True):
        href = a["href"].strip()
        urls.add(href)
        # # print(href)

        # # Convert relative URLs to absolute
        # full_url = urljoin(base_url, href)

        # parsed = urlparse(full_url)

        # # Check domain
        # if parsed.netloc == "https://www.stonerealestate.com.au":
        #     urls.add(full_url)

    return sorted(urls)

urls = get_stone_realestate_urls(soup)

for url in urls:
    print(url)

#
http://thestoneshop.com.au/
https://stone.lightspeedvt.com/
https://stone.supplycrew.com.au/
https://www.agentbox.com.au/features-pages/websites
https://www.facebook.com/pages/Stone-Real-Estate/123695347694910
https://www.instagram.com/STONE_Real_Estate/
https://www.linkedin.com/company/stone-real-estate-?trk=tyah&trkInfo=clickedVertical%3Acompany%2CclickedEntityId%3A6405155%2Cidx%3A3-2-7%2CtarId%3A1450153664292%2Ctas%3AStone+real+estate
https://www.lookatmystrata.com.au/login/966851
https://www.stonerealestate.com.au
https://www.stonerealestate.com.au/2022-real-estate-market-forecast/
https://www.stonerealestate.com.au/3-reasons-you-should-use-indoor-plants-to-bring-your-living-spaces-to-life/
https://www.stonerealestate.com.au/5-tips-how-to-style-every-room-in-your-home-this-christmas/
https://www.stonerealestate.com.au/a-5-step-survival-guide-for-hosting-christmas/
https://www.stonerealestate.com.au/a-three-step-guide-to-buying-your-first-investment-property/
https://www.stonereal

In [ ]:
url = "https://www.stonerealestate.com.au/wp-content/plugins/zoorealty/display/elements/load_listings.php?post_id=116&status=1,4&list=ResidentialSale&order=&order_direction=&property_type=&keywords=&price_min=&price_max=&bedrooms=&bathrooms=&carspaces=&sur_suburbs=&related_office=&office_id=&exclude_office_id=&page=17"
res = requests.get(url)
res.text

'\t\r\n\t\t\t<div class="column listing">\r\n\t\r\n\t\t\t\t<div class="container">\r\n\t\r\n\t\t\t\t\t<div class="image">\r\n\t\t\t\t\t\t<a class="centerimage" href="https://www.stonerealestate.com.au/property/8566886-16-1684-channel-highway-margate-tas/" title="Margate 16/1684 Channel Highway">\r\n\t\t\t\t\t\t\t<img class="fit-height" src="https://phimg.reapit.website/bfdc4d00f9a79f51d5c09c0fc195d2ea693676d8" alt="Margate 16/1684 Channel Highway" />\r\n\t\t\t\t\t\t</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t<div class="sticker underoffer"  style="display:block" >Under offer</div>\r\n\t\t\t\t\t\t\t\t\t\t\t</div>\r\n\t\r\n\t\t\t\t\t<div class="details">\r\n\t\t\r\n\t\t\t\t\t\t<div class="title">\r\n\t\t\t\t\t\t\t<p class="suburb"><a href="https://www.stonerealestate.com.au/property/8566886-16-1684-channel-highway-margate-tas/" title="Margate">Margate</a>&nbsp;</p>\r\n\t\t\t\t\t\t\t<p class="address">16/1684 Channel Highway&nbsp;</p>\r\n\t\t\t\t\t\t</div>\r\n\t\t\r\n\t\t\t\t\t\t<p class="rooms">\r

In [ ]:
from bs4 import BeautifulSoup

def parse_property_listings(soup):
    listings = []

    for card in soup.select("div.column.listing"):
        data = {}

        # --- URL & Image ---
        link = card.select_one(".image a.centerimage")
        data["url"] = link["href"] if link else None

        img = card.select_one(".image img")
        data["image_url"] = img["src"] if img else None

        # --- Status (Under offer etc.) ---
        sticker = card.select_one(".sticker")
        data["status"] = sticker.get_text(strip=True) if sticker else None

        # --- Location ---
        data["suburb"] = (
            card.select_one(".title .suburb a").get_text(strip=True)
            if card.select_one(".title .suburb a")
            else None
        )

        data["address"] = (
            card.select_one(".title .address").get_text(strip=True)
            if card.select_one(".title .address")
            else None
        )

        # --- Rooms ---
        rooms = card.select(".rooms .item")
        data["bedrooms"] = None
        data["bathrooms"] = None
        data["carspaces"] = None
        data["has_study"] = False

        for r in rooms:
            text = r.get_text(strip=True)
            icon = r.select_one("i")

            if not icon:
                continue

            cls = " ".join(icon.get("class", []))

            if "icon-bedrooms" in cls:
                data["bedrooms"] = int(text.split()[0])
            elif "icon-bathrooms" in cls:
                data["bathrooms"] = int(text.split()[0])
            elif "icon-carspaces" in cls:
                data["carspaces"] = int(text.split()[0])
            elif "icon-study" in cls:
                data["has_study"] = True

        # --- Price & Open Times ---
        price_block = card.select_one(".price")

        data["price"] = None
        data["open_times"] = None

        if price_block:
            default_price = price_block.select_one(".default-price")
            if default_price:
                data["price"] = default_price.get_text(strip=True)

            open_amount = price_block.select_one(".opens-price .amount")
            if open_amount:
                open_text = open_amount.get_text(strip=True)
                if open_text:
                    data["open_times"] = open_text

        listings.append(data)

    return listings

url = "https://www.stonerealestate.com.au/wp-content/plugins/zoorealty/display/elements/load_listings.php?post_id=116&status=1,4&list=ResidentialSale&order=&order_direction=&property_type=&keywords=&price_min=&price_max=&bedrooms=&bathrooms=&carspaces=&sur_suburbs=&related_office=&office_id=&exclude_office_id=&page=17"
res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")
properties = parse_property_listings(soup)

for p in properties:
    print(p)

{'url': 'https://www.stonerealestate.com.au/property/8566886-16-1684-channel-highway-margate-tas/', 'image_url': 'https://phimg.reapit.website/bfdc4d00f9a79f51d5c09c0fc195d2ea693676d8', 'status': 'Under offer', 'suburb': 'Margate', 'address': '16/1684 Channel Highway', 'bedrooms': 2, 'bathrooms': 1, 'carspaces': 2, 'has_study': False, 'price': 'Offers over $525,000', 'open_times': None}
{'url': 'https://www.stonerealestate.com.au/property/8566857-10-1-3-wentworth-street-manly-nsw/', 'image_url': 'https://phimg.reapit.website/6c6a00f77695dc14f3891d4d005bad87ad8a2159', 'status': None, 'suburb': 'Manly', 'address': '10/1-3 Wentworth Street', 'bedrooms': 2, 'bathrooms': 2, 'carspaces': 2, 'has_study': False, 'price': 'Auction', 'open_times': None}
{'url': 'https://www.stonerealestate.com.au/property/8566856-4-8-james-street-manly-nsw/', 'image_url': 'https://phimg.reapit.website/6732b07574cfee6e6b9cf7d462bc1f9c3b7f5e19', 'status': None, 'suburb': 'Manly', 'address': '4/8 James Street', 'be

In [ ]:
import requests
from bs4 import BeautifulSoup

BASE_URL = (
    "https://www.stonerealestate.com.au/wp-content/plugins/zoorealty/"
    "display/elements/load_listings.php"
)

PARAMS = {
    "post_id": "1963",
    "status": "1,4",
    "list": "CommercialLease",
    "order": "",
    "order_direction": "",
    "property_type": "",
    "keywords": "",
    "price_min": "",
    "price_max": "",
    "bedrooms": "",
    "bathrooms": "",
    "carspaces": "",
    "sur_suburbs": "",
    "related_office": "",
    "office_id": "",
    "exclude_office_id": "",
}

all_properties = []
page = 1

while True:
    PARAMS["page"] = page

    print(f"Fetching page {page}...")

    res = requests.get(BASE_URL, params=PARAMS, timeout=15)
    res.raise_for_status()

    soup = BeautifulSoup(res.text, "html.parser")
    properties = parse_property_listings(soup)

    # Stop if no properties returned
    if not properties:
        print("No properties found, stopping.")
        break

    all_properties.extend(properties)

    print(f"→ Found {len(properties)} properties")

    # Stop condition: last page has fewer than 12 items
    if len(properties) < 12:
        print("Last page reached.")
        break

    page += 1

print(f"\nTotal properties collected: {len(all_properties)}")


Fetching page 1...
→ Found 12 properties
Fetching page 2...
→ Found 12 properties
Fetching page 3...
→ Found 12 properties
Fetching page 4...
→ Found 12 properties
Fetching page 5...
→ Found 12 properties
Fetching page 6...
→ Found 12 properties
Fetching page 7...
→ Found 5 properties
Last page reached.

Total properties collected: 77


In [ ]:
import json

output_filename = 'CommercialLease.json'
with open(output_filename, 'w') as f:
    json.dump(all_properties, f, indent=4)

print(f"Successfully saved {len(all_properties)} properties to {output_filename}")

Successfully saved 77 properties to CommercialLease.json


In [ ]:
# @title
pages = [
  "https://www.stonerealestate.com.au",
  "https://www.stonerealestate.com.au/about/",
  "https://www.stonerealestate.com.au/acreage/",
  "https://www.stonerealestate.com.au/blog/",
  "https://www.stonerealestate.com.au/buy/",
  "https://www.stonerealestate.com.au/buy/acreage/",
  "https://www.stonerealestate.com.au/buy/commercial/",
  "https://www.stonerealestate.com.au/buy/off-market-sales/",
  "https://www.stonerealestate.com.au/buy/open-sales/",
  "https://www.stonerealestate.com.au/buy/property-alerts/",
  "https://www.stonerealestate.com.au/buy/residential/",
  "https://www.stonerealestate.com.au/careers/",
  "https://www.stonerealestate.com.au/franchise/",
  "https://www.stonerealestate.com.au/manage/",
  "https://www.stonerealestate.com.au/our-approach-to-management/",
  "https://www.stonerealestate.com.au/our-approach-to-selling/",
  "https://www.stonerealestate.com.au/our-people/join-us/",
  "https://www.stonerealestate.com.au/our-people/our-agents/",
  "https://www.stonerealestate.com.au/our-people/our-offices/",
  "https://www.stonerealestate.com.au/privacy-policy/",
  "https://www.stonerealestate.com.au/projects/",
  "https://www.stonerealestate.com.au/property-report/",
  "https://www.stonerealestate.com.au/property/4394568-18-arrowgrass-street-aberglasslyn-nsw/",
  "https://www.stonerealestate.com.au/property/5028950-5-foxtail-street-fern-bay-nsw/",
  "https://www.stonerealestate.com.au/property/6412043-369a-bexley-road-bexley-nsw/",
  "https://www.stonerealestate.com.au/property/6757984-56a-geoffrey-road-chittaway-point-nsw/",
  "https://www.stonerealestate.com.au/property/7155103-38-nicoll-street-roselands-nsw/",
  "https://www.stonerealestate.com.au/property/7336529-1-21-heyes-street-gillieston-heights-nsw/",
  "https://www.stonerealestate.com.au/property/7348819-17-10-16-hegerty-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/7430147-81a-melbourne-street-aberdare-nsw/",
  "https://www.stonerealestate.com.au/property/7556158-2-muirfield-way-medowie-nsw/",
  "https://www.stonerealestate.com.au/property/7694555-3-64-66-connemarra-street-bexley-nsw/",
  "https://www.stonerealestate.com.au/property/7729937-18-3-gahnia-place-hamlyn-terrace-nsw/",
  "https://www.stonerealestate.com.au/property/7796595-1-14-aspect-drive-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/7807294-4-4-rocklands-road-wollstonecraft-nsw/",
  "https://www.stonerealestate.com.au/property/7845916-3316-2-rothschild-avenue-rosebery-nsw/",
  "https://www.stonerealestate.com.au/property/7968305-34-trevally-street-korora-nsw/",
  "https://www.stonerealestate.com.au/property/7982724-92-94-talavera-road-macquarie-park-nsw/",
  "https://www.stonerealestate.com.au/property/8009527-7-28-marine-parade-miami-qld/",
  "https://www.stonerealestate.com.au/property/8046624-75-amalfi-drive-isle-of-capri-qld/",
  "https://www.stonerealestate.com.au/property/8083353-44-1-9-beach-street-the-entrance-nsw/",
  "https://www.stonerealestate.com.au/property/8213526-1-4-10-the-boulevarde-brighton-le-sands-nsw/",
  "https://www.stonerealestate.com.au/property/8223674-9a-chelsea-street-balmoral-nsw/",
  "https://www.stonerealestate.com.au/property/8231572-31-chico-street-copacabana-nsw/",
  "https://www.stonerealestate.com.au/property/8248230-4b-earl-street-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/8252194-56-the-corso-gorokan-nsw/",
  "https://www.stonerealestate.com.au/property/8265745-3-5-pitt-owen-avenue-arncliffe-nsw/",
  "https://www.stonerealestate.com.au/property/8266979-24-table-top-road-north-avoca-nsw/",
  "https://www.stonerealestate.com.au/property/8292238-196-chatham-street-hamilton-south-nsw/",
  "https://www.stonerealestate.com.au/property/8298945-51-harbourside-crescent-port-macquarie-nsw/",
  "https://www.stonerealestate.com.au/property/8304850-242-252-pitt-street-merrylands-nsw/",
  "https://www.stonerealestate.com.au/property/8305600-23a-frances-street-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/8321749-155-riverview-street-riverview-nsw/",
  "https://www.stonerealestate.com.au/property/8359142-1-113-oceano-street-copacabana-nsw/",
  "https://www.stonerealestate.com.au/property/8376018-2-driver-place-wyong-nsw/",
  "https://www.stonerealestate.com.au/property/8383586-2-25-29-oconnell-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8384670-1-45-murdoch-street-cremorne-nsw/",
  "https://www.stonerealestate.com.au/property/8385953-26-peterborough-avenue-lake-illawarra-nsw/",
  "https://www.stonerealestate.com.au/property/8387628-309-38-44-pembroke-street-epping-nsw/",
  "https://www.stonerealestate.com.au/property/8393917-1-strangers-creek-place-bella-vista-nsw/",
  "https://www.stonerealestate.com.au/property/8393918-21-strangers-creek-place-bella-vista-nsw/",
  "https://www.stonerealestate.com.au/property/8393919-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8402928-204-8-12-good-street-westmead-nsw/",
  "https://www.stonerealestate.com.au/property/8402929-205-8-12-good-street-westmead-nsw/",
  "https://www.stonerealestate.com.au/property/8413793-9-kateena-avenue-tascott-nsw/",
  "https://www.stonerealestate.com.au/property/8420151-34-brett-place-west-pennant-hills-nsw/",
  "https://www.stonerealestate.com.au/property/8420222-413-yarramalong-road-wyong-creek-nsw/",
  "https://www.stonerealestate.com.au/property/8420427-95-aloha-drive-chittaway-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8439122-1a-thomas-mitchell-road-killarney-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8445435-3604-11-hassall-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8445542-1-22-24-wagawn-street-tugun-qld/",
  "https://www.stonerealestate.com.au/property/8446755-36-lennox-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/8447566-7b-alicia-road-mount-kuring-gai-nsw/",
  "https://www.stonerealestate.com.au/property/8449656-37b-griffiths-street-sans-souci-nsw/",
  "https://www.stonerealestate.com.au/property/8449690-210-25-mann-street-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8451176-64-buff-point-avenue-buff-point-nsw/",
  "https://www.stonerealestate.com.au/property/8452993-204-10-west-promenade-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8454332-8-mcmaster-place-little-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8456581-5-35-scenic-highway-terrigal-nsw/",
  "https://www.stonerealestate.com.au/property/8458858-34-montego-court-eatons-hill-qld/",
  "https://www.stonerealestate.com.au/property/8462727-23-ponsonby-parade-seaforth-nsw/",
  "https://www.stonerealestate.com.au/property/8465008-658-15-thompsons-road-pokolbin-nsw/",
  "https://www.stonerealestate.com.au/property/8465561-g21-4-seven-street-epping-nsw/",
  "https://www.stonerealestate.com.au/property/8465829-30-dampier-boulevard-killarney-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8465842-3-32-dening-street-the-entrance-nsw/",
  "https://www.stonerealestate.com.au/property/8469612-88-katherine-avenue-amaroo-act/",
  "https://www.stonerealestate.com.au/property/8470970-16-lampard-circuit-bruce-act/",
  "https://www.stonerealestate.com.au/property/8471682-37-dunmore-street-north-bexley-nsw/",
  "https://www.stonerealestate.com.au/property/8472024-59-burdekin-avenue-amaroo-act/",
  "https://www.stonerealestate.com.au/property/8472332-206-208-oak-road-matcham-nsw/",
  "https://www.stonerealestate.com.au/property/8472472-16-colblack-close-tacoma-nsw/",
  "https://www.stonerealestate.com.au/property/8475837-94-highgate-street-bexley-nsw/",
  "https://www.stonerealestate.com.au/property/8480334-87-mabo-boulevard-bonner-act/",
  "https://www.stonerealestate.com.au/property/8480520-9-12-linda-street-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8480894-70a-wattle-tree-road-holgate-nsw/",
  "https://www.stonerealestate.com.au/property/8481846-16-14-jenkins-street-collaroy-nsw/",
  "https://www.stonerealestate.com.au/property/8482037-8-jasmine-street-horsley-nsw/",
  "https://www.stonerealestate.com.au/property/8491007-23-fishermans-parade-daleys-point-nsw/",
  "https://www.stonerealestate.com.au/property/8492181-1-2-pape-street-franklin-act/",
  "https://www.stonerealestate.com.au/property/8492576-1001-16-18-seventh-avenue-palm-beach-qld/",
  "https://www.stonerealestate.com.au/property/8493350-55b-high-street-thirroul-nsw/",
  "https://www.stonerealestate.com.au/property/8493709-2-3-7-gladstone-street-north-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8493718-302-107-chandos-street-crows-nest-nsw/",
  "https://www.stonerealestate.com.au/property/8494031-60-spinnaker-ridge-way-belmont-nsw/",
  "https://www.stonerealestate.com.au/property/8494831-40-134-shoalhaven-street-kiama-nsw/",
  "https://www.stonerealestate.com.au/property/8494983-7-judith-street-seaforth-nsw/",
  "https://www.stonerealestate.com.au/property/8495130-2-bracken-close-berowra-nsw/",
  "https://www.stonerealestate.com.au/property/8496877-305-77-edward-street-carlton-nsw/",
  "https://www.stonerealestate.com.au/property/8496937-369-lakedge-avenue-berkeley-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8497047-30-ladera-drive-copacabana-nsw/",
  "https://www.stonerealestate.com.au/property/8497173-6-leopard-avenue-elanora-qld/",
  "https://www.stonerealestate.com.au/property/8499347-1-boomerang-street-lake-cathie-nsw/",
  "https://www.stonerealestate.com.au/property/8500373-6-68-smith-street-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8502183-40-zoriana-close-mardi-nsw/",
  "https://www.stonerealestate.com.au/property/8502655-26-quealy-street-casey-act/",
  "https://www.stonerealestate.com.au/property/8503773-46-2-brunswick-road-terrigal-nsw/",
  "https://www.stonerealestate.com.au/property/8503794-213-414-pacific-highway-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8504328-23-murray-road-beecroft-nsw/",
  "https://www.stonerealestate.com.au/property/8504335-7-sheoak-drive-buninyong-vic/",
  "https://www.stonerealestate.com.au/property/8505514-5-guyong-street-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8506386-22-18-20-market-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/8506462-13-22-zelman-close-watanobbi-nsw/",
  "https://www.stonerealestate.com.au/property/8506661-16-curtis-parade-the-entrance-north-nsw/",
  "https://www.stonerealestate.com.au/property/8506747-283-lakedge-avenue-berkeley-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8509374-5-66-68-park-road-rydalmere-nsw/",
  "https://www.stonerealestate.com.au/property/8509651-5-amadeus-place-north-boambee-valley-nsw/",
  "https://www.stonerealestate.com.au/property/8509836-12-pinehurst-avenue-rouse-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8509888-1329-100-belmore-street-ryde-nsw/",
  "https://www.stonerealestate.com.au/property/8509980-21-brian-street-fairy-meadow-nsw/",
  "https://www.stonerealestate.com.au/property/8510162-3-10-armidale-street-the-entrance-nsw/",
  "https://www.stonerealestate.com.au/property/8510611-3-bolwarra-crescent-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8514302-7-amadeus-place-north-boambee-valley-nsw/",
  "https://www.stonerealestate.com.au/property/8515384-13-schuppan-street-oran-park-nsw/",
  "https://www.stonerealestate.com.au/property/8515394-level-3-311-2b-wharf-road-melrose-park-nsw/",
  "https://www.stonerealestate.com.au/property/8515593-130-lakedge-avenue-berkeley-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8516132-1-ralston-avenue-belrose-nsw/",
  "https://www.stonerealestate.com.au/property/8517253-15-karoo-avenue-east-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8517335-2-bromborough-road-roseville-nsw/",
  "https://www.stonerealestate.com.au/property/8517610-33-oratava-avenue-west-pennant-hills-nsw/",
  "https://www.stonerealestate.com.au/property/8517612-204-7-gladstone-parade-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8517853-3-windsorgreen-drive-wyong-nsw/",
  "https://www.stonerealestate.com.au/property/8518133-27-spinks-road-east-corrimal-nsw/",
  "https://www.stonerealestate.com.au/property/8519075-12-83-85-alfred-street-ramsgate-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8520565-29-kirby-street-rydalmere-nsw/",
  "https://www.stonerealestate.com.au/property/8520865-102-seaforth-crescent-seaforth-nsw/",
  "https://www.stonerealestate.com.au/property/8521599-8a-arkana-street-telopea-nsw/",
  "https://www.stonerealestate.com.au/property/8522646-8-endeavour-street-sans-souci-nsw/",
  "https://www.stonerealestate.com.au/property/8522718-9-2-6-bowen-street-chatswood-nsw/",
  "https://www.stonerealestate.com.au/property/8523144-101-hills-street-north-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8523342-58-scenic-highway-terrigal-nsw/",
  "https://www.stonerealestate.com.au/property/8523353-114-40-cotlew-street-east-southport-qld/",
  "https://www.stonerealestate.com.au/property/8524184-42-41-chandler-street-belconnen-act/",
  "https://www.stonerealestate.com.au/property/8524471-12-94-henry-kendall-street-franklin-act/",
  "https://www.stonerealestate.com.au/property/8526151-27-bath-street-thirroul-nsw/",
  "https://www.stonerealestate.com.au/property/8526404-39-bronhill-street-moncrieff-act/",
  "https://www.stonerealestate.com.au/property/8526810-31-cambridge-avenue-windsor-nsw/",
  "https://www.stonerealestate.com.au/property/8527649-33-orana-avenue-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8528553-246-bathurst-street-sawyers-gully-nsw/",
  "https://www.stonerealestate.com.au/property/8528856-22-15-21-webb-avenue-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8528861-6-61-broken-bay-road-ettalong-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8529370-14-tamora-close-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/8530350-202b-3-11-burleigh-street-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8530674-14-anvil-avenue-elderslie-nsw/",
  "https://www.stonerealestate.com.au/property/8531251-8-wolseley-road-mcgraths-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8531335-12a-gosford-avenue-the-entrance-nsw/",
  "https://www.stonerealestate.com.au/property/8531560-12-470-472-pacific-highway-asquith-nsw/",
  "https://www.stonerealestate.com.au/property/8531714-120-gooraway-drive-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8532044-5-2-6-russell-avenue-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8532241-192-parraweena-road-miranda-nsw/",
  "https://www.stonerealestate.com.au/property/8534268-41-19-23-herbert-street-st-leonards-nsw/",
  "https://www.stonerealestate.com.au/property/8536176-5-ray-ellis-crescent-forde-act/",
  "https://www.stonerealestate.com.au/property/8536177-28-hedland-circuit-flynn-act/",
  "https://www.stonerealestate.com.au/property/8536354-60-west-burleigh-road-burleigh-heads-qld/",
  "https://www.stonerealestate.com.au/property/8536505-352-west-botany-street-brighton-le-sands-nsw/",
  "https://www.stonerealestate.com.au/property/8537005-12-greta-road-kulnura-nsw/",
  "https://www.stonerealestate.com.au/property/8537338-53-north-avoca-parade-north-avoca-nsw/",
  "https://www.stonerealestate.com.au/property/8537993-10-blaxland-road-killara-nsw/",
  "https://www.stonerealestate.com.au/property/8538000-1-west-hill-street-mcgraths-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8539189-37-hull-road-beecroft-nsw/",
  "https://www.stonerealestate.com.au/property/8539190-22-49-55-beamish-road-northmead-nsw/",
  "https://www.stonerealestate.com.au/property/8539704-9-phelp-court-amaroo-act/",
  "https://www.stonerealestate.com.au/property/8539706-4-wardell-place-agnes-banks-nsw/",
  "https://www.stonerealestate.com.au/property/8539757-7-jule-knight-street-casey-act/",
  "https://www.stonerealestate.com.au/property/8539802-4-benaroon-circuit-amaroo-act/",
  "https://www.stonerealestate.com.au/property/8539850-47a-malvern-avenue-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8539986-20-klaus-moje-street-whitlam-act/",
  "https://www.stonerealestate.com.au/property/8540182-109-quarry-road-rosewood-nsw/",
  "https://www.stonerealestate.com.au/property/8540760-14-62-lowanna-street-braddon-act/",
  "https://www.stonerealestate.com.au/property/8541226-62-orchard-road-kangy-angy-nsw/",
  "https://www.stonerealestate.com.au/property/8541684-18-134-shoalhaven-street-kiama-nsw/",
  "https://www.stonerealestate.com.au/property/8541745-9-amadeus-place-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/8543302-14-bligh-street-silverwater-nsw/",
  "https://www.stonerealestate.com.au/property/8543511-144-macquarie-grove-caves-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8543601-82-malton-road-beecroft-nsw/",
  "https://www.stonerealestate.com.au/property/8544090-360-lower-colo-road-lower-portland-nsw/",
  "https://www.stonerealestate.com.au/property/8544294-7-reid-street-croki-nsw/",
  "https://www.stonerealestate.com.au/property/8544295-682-ocean-drive-north-haven-nsw/",
  "https://www.stonerealestate.com.au/property/8544365-326-blaxlands-ridge-road-blaxlands-ridge-nsw/",
  "https://www.stonerealestate.com.au/property/8544445-11-bogan-road-booker-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8544773-30-applecross-avenue-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8544873-176-geoffrey-road-chittaway-point-nsw/",
  "https://www.stonerealestate.com.au/property/8545204-11-dandarbong-avenue-carlingford-nsw/",
  "https://www.stonerealestate.com.au/property/8545424-2-522-524-pacific-highway-mount-colah-nsw/",
  "https://www.stonerealestate.com.au/property/8545547-33-st-johns-road-bradbury-nsw/",
  "https://www.stonerealestate.com.au/property/8545618-48-amiens-street-gladesville-nsw/",
  "https://www.stonerealestate.com.au/property/8545720-9-martins-creek-road-paterson-nsw/",
  "https://www.stonerealestate.com.au/property/8546189-7-43-47-mallett-street-camperdown-nsw/",
  "https://www.stonerealestate.com.au/property/8546291-64-71-83-smith-street-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8546395-43-primula-street-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8546887-39-bayview-street-bexley-nsw/",
  "https://www.stonerealestate.com.au/property/8546891-4-hooton-street-forde-act/",
  "https://www.stonerealestate.com.au/property/8547175-10-52-bay-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/8547346-27-little-lake-crescent-warilla-nsw/",
  "https://www.stonerealestate.com.au/property/8547390-8-tippet-place-quakers-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8547394-12-philip-charley-drive-port-macquarie-nsw/",
  "https://www.stonerealestate.com.au/property/8547539-31-connaghan-avenue-east-corrimal-nsw/",
  "https://www.stonerealestate.com.au/property/8547597-425-doveton-street-soldiers-hill-vic/",
  "https://www.stonerealestate.com.au/property/8548349-88b-howes-road-east-kurrajong-nsw/",
  "https://www.stonerealestate.com.au/property/8548702-317-43-arthur-blakeley-way-coombs-act/",
  "https://www.stonerealestate.com.au/property/8551197-198-victoria-avenue-chatswood-nsw/",
  "https://www.stonerealestate.com.au/property/8551210-88-tallebudgera-drive-palm-beach-qld/",
  "https://www.stonerealestate.com.au/property/8551644-34-1-nevertire-street-lawson-act/",
  "https://www.stonerealestate.com.au/property/8552028-241-williams-road-kulnura-nsw/",
  "https://www.stonerealestate.com.au/property/8552804-35-albert-drive-killara-nsw/",
  "https://www.stonerealestate.com.au/property/8552870-204-10-mentmore-avenue-rosebery-nsw/",
  "https://www.stonerealestate.com.au/property/8553188-110-8-baudinette-circuit-bruce-act/",
  "https://www.stonerealestate.com.au/property/8553263-4-53-east-esplanade-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8553937-1-sickles-drive-grasmere-nsw/",
  "https://www.stonerealestate.com.au/property/8554041-404-wendouree-parade-lake-wendouree-vic/",
  "https://www.stonerealestate.com.au/property/8554111-4-harwood-avenue-mount-kuring-gai-nsw/",
  "https://www.stonerealestate.com.au/property/8554183-1-32-dening-street-the-entrance-nsw/",
  "https://www.stonerealestate.com.au/property/8554194-204-39-george-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/8554303-86-kalua-drive-chittaway-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8554388-25-david-street-cessnock-nsw/",
  "https://www.stonerealestate.com.au/property/8554510-63-christopher-avenue-valentine-nsw/",
  "https://www.stonerealestate.com.au/property/8554626-18-warrawilla-road-wyoming-nsw/",
  "https://www.stonerealestate.com.au/property/8554915-level-10-1045-1-finch-drive-eastgardens-nsw/",
  "https://www.stonerealestate.com.au/property/8555320-13-23-glenvale-close-west-pennant-hills-nsw/",
  "https://www.stonerealestate.com.au/property/8555830-11-10-kalinya-street-newport-nsw/",
  "https://www.stonerealestate.com.au/property/8555914-79-hirst-street-arncliffe-nsw/",
  "https://www.stonerealestate.com.au/property/8555917-19-malcolm-cole-terrace-whitlam-act/",
  "https://www.stonerealestate.com.au/property/8556091-29-alana-drive-west-pennant-hills-nsw/",
  "https://www.stonerealestate.com.au/property/8556623-5-25-waine-street-freshwater-nsw/",
  "https://www.stonerealestate.com.au/property/8556801-14-10-14-bruce-street-brighton-le-sands-nsw/",
  "https://www.stonerealestate.com.au/property/8556887-28-edith-street-cessnock-nsw/",
  "https://www.stonerealestate.com.au/property/8557548-51-werona-avenue-gordon-nsw/",
  "https://www.stonerealestate.com.au/property/8557740-14-andrew-thompson-drive-mcgraths-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8557805-39-bruce-dittmar-street-forde-act/",
  "https://www.stonerealestate.com.au/property/8557872-25-240-ben-boyd-road-cremorne-nsw/",
  "https://www.stonerealestate.com.au/property/8557873-11-174-spit-road-mosman-nsw/",
  "https://www.stonerealestate.com.au/property/8557948-84-whalans-road-greystanes-nsw/",
  "https://www.stonerealestate.com.au/property/8557951-33-ross-street-windsor-nsw/",
  "https://www.stonerealestate.com.au/property/8558152-1401a-1-17-elsie-street-burwood-nsw/",
  "https://www.stonerealestate.com.au/property/8558193-4-gilgandra-street-lake-heights-nsw/",
  "https://www.stonerealestate.com.au/property/8559215-4-74-80-woniora-road-hurstville-nsw/",
  "https://www.stonerealestate.com.au/property/8559222-10-riversford-close-menangle-nsw/",
  "https://www.stonerealestate.com.au/property/8559307-62-tindara-drive-sawtell-nsw/",
  "https://www.stonerealestate.com.au/property/8559535-13-cousins-road-beacon-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8559770-47-jorgensen-street-moncrieff-act/",
  "https://www.stonerealestate.com.au/property/8559771-60-pademelon-street-throsby-act/",
  "https://www.stonerealestate.com.au/property/8560008-4b-shore-break-crescent-lake-cathie-nsw/",
  "https://www.stonerealestate.com.au/property/8560068-35a-king-street-manly-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8560088-2903-330-church-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8560189-6a-tessa-street-chatswood-nsw/",
  "https://www.stonerealestate.com.au/property/8560551-7-688-rocky-point-road-sans-souci-nsw/",
  "https://www.stonerealestate.com.au/property/8560557-23-tunbridge-place-jannali-nsw/",
  "https://www.stonerealestate.com.au/property/8560810-10-castle-court-kallangur-qld/",
  "https://www.stonerealestate.com.au/property/8561040-9-gleneagles-avenue-killara-nsw/",
  "https://www.stonerealestate.com.au/property/8561042-35-505-507-rocky-point-road-sans-souci-nsw/",
  "https://www.stonerealestate.com.au/property/8561047-42-mangrove-road-narara-nsw/",
  "https://www.stonerealestate.com.au/property/8561195-36-24-flinders-street-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8561349-28-kurrawyba-avenue-terrigal-nsw/",
  "https://www.stonerealestate.com.au/property/8561577-371-newport-road-cooranbong-nsw/",
  "https://www.stonerealestate.com.au/property/8561679-8-reef-street-nirimba-fields-quakers-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8561749-33-adelaide-street-balgowlah-heights-nsw/",
  "https://www.stonerealestate.com.au/property/8561954-4a-low-street-mount-kuring-gai-nsw/",
  "https://www.stonerealestate.com.au/property/8562124-10-narramore-street-kingsgrove-nsw/",
  "https://www.stonerealestate.com.au/property/8562384-2-aria-entrance-rothbury-nsw/",
  "https://www.stonerealestate.com.au/property/8562388-29-penfold-street-gungahlin-act/",
  "https://www.stonerealestate.com.au/property/8562389-12-miago-court-ngunnawal-act/",
  "https://www.stonerealestate.com.au/property/8562776-60-hawkesbury-drive-donnybrook-vic/",
  "https://www.stonerealestate.com.au/property/8562845-88-burns-road-kellyville-nsw/",
  "https://www.stonerealestate.com.au/property/8562923-12-pheasant-street-toronto-nsw/",
  "https://www.stonerealestate.com.au/property/8563114-4-raymond-street-tacoma-south-nsw/",
  "https://www.stonerealestate.com.au/property/8563690-3-19-oak-circuit-raymond-terrace-nsw/",
  "https://www.stonerealestate.com.au/property/8563880-374-mandalay-circuit-beveridge-vic/",
  "https://www.stonerealestate.com.au/property/8564239-45-burgess-street-beverley-park-nsw/",
  "https://www.stonerealestate.com.au/property/8564681-4-2-factory-street-north-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8564754-6-elizabeth-street-riverstone-nsw/",
  "https://www.stonerealestate.com.au/property/8564979-3-brierley-place-eagle-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8565052-6-111-harbord-road-freshwater-nsw/",
  "https://www.stonerealestate.com.au/property/8565166-2-21-heyes-street-gillieston-heights-nsw/",
  "https://www.stonerealestate.com.au/property/8565182-40-brabyn-street-north-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8565241-16-clark-road-noraville-nsw/",
  "https://www.stonerealestate.com.au/property/8565250-185a-cumberland-road-greystanes-nsw/",
  "https://www.stonerealestate.com.au/property/8565291-70-dunning-avenue-rosebery-nsw/",
  "https://www.stonerealestate.com.au/property/8565326-654-ourimbah-creek-road-palm-grove-nsw/",
  "https://www.stonerealestate.com.au/property/8565737-24-9-banksia-avenue-banksia-nsw/",
  "https://www.stonerealestate.com.au/property/8565937-37-bundilla-parade-berkeley-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8565938-103-70-efkarpidis-street-gungahlin-act/",
  "https://www.stonerealestate.com.au/property/8566017-25-chippindall-street-speers-point-nsw/",
  "https://www.stonerealestate.com.au/property/8566742-97-rebecca-parade-winston-hills-nsw/",
  "https://www.stonerealestate.com.au/property/8566795-3-17-buckland-street-balgownie-nsw/",
  "https://www.stonerealestate.com.au/property/8566796-9-saltwater-circuit-kanahooka-nsw/",
  "https://www.stonerealestate.com.au/property/8566856-4-8-james-street-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8566857-10-1-3-wentworth-street-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8566864-4-35-37-ross-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8567063-30-972-990-old-princes-highway-engadine-nsw/",
  "https://www.stonerealestate.com.au/property/8567064-33-oakland-avenue-the-entrance-nsw/",
  "https://www.stonerealestate.com.au/property/8567209-46-robsons-road-keiraville-nsw/",
  "https://www.stonerealestate.com.au/property/8567274-3-45-cessnock-road-weston-nsw/",
  "https://www.stonerealestate.com.au/property/8567341-5-23-great-western-highway-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8567830-40-mcgee-avenue-wamberal-nsw/",
  "https://www.stonerealestate.com.au/property/8567890-101-aloha-drive-chittaway-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8567976-22-daley-loop-harrington-park-nsw/",
  "https://www.stonerealestate.com.au/property/8568063-20a-arlington-street-gorokan-nsw/",
  "https://www.stonerealestate.com.au/property/8568126-25-kirkwall-avenue-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8568283-level-4-33-2a-brady-street-mosman-nsw/",
  "https://www.stonerealestate.com.au/property/8568402-704-23-george-street-burwood-nsw/",
  "https://www.stonerealestate.com.au/property/8568727-123-koloona-avenue-mount-keira-nsw/",
  "https://www.stonerealestate.com.au/property/8568766-15-woods-street-redhead-nsw/",
  "https://www.stonerealestate.com.au/property/8568799-47-harrow-circuit-greta-nsw/",
  "https://www.stonerealestate.com.au/property/8569119-802-36-baker-street-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8569593-43-235-flemington-road-franklin-act/",
  "https://www.stonerealestate.com.au/property/8569736-102-50-54-mallett-street-camperdown-nsw/",
  "https://www.stonerealestate.com.au/property/8569760-40-mills-crescent-cessnock-nsw/",
  "https://www.stonerealestate.com.au/property/8569779-22-pindari-drive-dunbogan-nsw/",
  "https://www.stonerealestate.com.au/property/8569783-100-atchison-street-crows-nest-nsw/",
  "https://www.stonerealestate.com.au/property/8569846-3-13-margaret-street-belmont-north-nsw/",
  "https://www.stonerealestate.com.au/property/8570002-803-20-kendall-street-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8570070-55-byron-court-phillip-act/",
  "https://www.stonerealestate.com.au/property/8570438-g1-39-moffatts-drive-dundas-valley-nsw/",
  "https://www.stonerealestate.com.au/property/8570454-9-katoomba-street-harrison-act/",
  "https://www.stonerealestate.com.au/property/8570526-7-silverdale-parade-jewells-nsw/",
  "https://www.stonerealestate.com.au/property/8570539-1a-wild-flower-place-tahmoor-nsw/",
  "https://www.stonerealestate.com.au/property/8570727-30-open-drive-snug-tas/",
  "https://www.stonerealestate.com.au/property/8571126-49-kings-canyon-street-harrison-act/",
  "https://www.stonerealestate.com.au/property/8571366-167-granite-street-port-macquarie-nsw/",
  "https://www.stonerealestate.com.au/property/8571402-20a-craigie-ave-kanwal-nsw/",
  "https://www.stonerealestate.com.au/property/8571693-24-261-condamine-street-manly-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8571697-8-wyong-road-tumbi-umbi-nsw/",
  "https://www.stonerealestate.com.au/property/8571735-17-campbell-street-north-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8571784-19-long-reef-crescent-woodbine-nsw/",
  "https://www.stonerealestate.com.au/property/8571876-30a-pass-avenue-thirroul-nsw/",
  "https://www.stonerealestate.com.au/property/8571893-34-rosella-street-nicholls-act/",
  "https://www.stonerealestate.com.au/property/8572041-7-mangrove-close-lakewood-nsw/",
  "https://www.stonerealestate.com.au/property/8572188-11-knight-street-arncliffe-nsw/",
  "https://www.stonerealestate.com.au/property/8572399-36-143-149-corrimal-street-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8572710-80-aspinall-street-watson-act/",
  "https://www.stonerealestate.com.au/property/8572770-25a-cohen-way-thrumster-nsw/",
  "https://www.stonerealestate.com.au/property/8572845-717-15-thompsons-road-pokolbin-nsw/",
  "https://www.stonerealestate.com.au/property/8572866-26-scarborough-way-dunbogan-nsw/",
  "https://www.stonerealestate.com.au/property/8572923-32-prince-street-bellbird-nsw/",
  "https://www.stonerealestate.com.au/property/8572945-ourimbah-nsw/",
  "https://www.stonerealestate.com.au/property/8572975-2-lewis-street-epping-nsw/",
  "https://www.stonerealestate.com.au/property/8573206-20-enright-drive-north-rothbury-nsw/",
  "https://www.stonerealestate.com.au/property/8573263-71-serafina-drive-helensvale-qld/",
  "https://www.stonerealestate.com.au/property/8573648-402-888b-pacific-highway-gordon-nsw/",
  "https://www.stonerealestate.com.au/property/8573713-206-100-bay-road-waverton-nsw/",
  "https://www.stonerealestate.com.au/property/8573877-38-peppercorn-avenue-narellan-nsw/",
  "https://www.stonerealestate.com.au/property/8574067-20-francis-street-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8574116-9-owen-avenue-kyeemagh-nsw/",
  "https://www.stonerealestate.com.au/property/8574530-14-popran-way-blue-haven-nsw/",
  "https://www.stonerealestate.com.au/property/8574601-17-47-victoria-road-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8574604-9-djerrkurra-street-bonner-act/",
  "https://www.stonerealestate.com.au/property/8574720-21-trinity-point-drive-morisset-park-nsw/",
  "https://www.stonerealestate.com.au/property/8574726-5-234-240-the-entrance-long-jetty-nsw/",
  "https://www.stonerealestate.com.au/property/8574728-17-plantation-place-avoca-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8574931-12-pioneer-street-amaroo-act/",
  "https://www.stonerealestate.com.au/property/8575016-25-166-russell-avenue-dolls-point-nsw/",
  "https://www.stonerealestate.com.au/property/8575019-12-11-15-villiers-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8575081-2-7-spectrum-road-north-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8575129-60-66-mclennans-road-plenty-vic/",
  "https://www.stonerealestate.com.au/property/8575162-184-mount-keira-road-mount-keira-nsw/",
  "https://www.stonerealestate.com.au/property/8575167-20-preddys-road-bexley-nsw/",
  "https://www.stonerealestate.com.au/property/8575384-5-6-parsonage-road-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8575385-13-29-31-muriel-street-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8575435-32-park-street-swansea-nsw/",
  "https://www.stonerealestate.com.au/property/8575543-66-23-regent-honeyeater-grv-north-kellyville-nsw/",
  "https://www.stonerealestate.com.au/property/8575618-28-carlisle-crescent-beecroft-nsw/",
  "https://www.stonerealestate.com.au/property/8575754-71-queen-street-narellan-nsw/",
  "https://www.stonerealestate.com.au/property/8575805-107c-80-waterloo-road-macquarie-park-nsw/",
  "https://www.stonerealestate.com.au/property/8575990-14-anglo-road-greenwich-nsw/",
  "https://www.stonerealestate.com.au/property/8576889-25-baikie-crescent-charlestown-nsw/",
  "https://www.stonerealestate.com.au/property/8577584-32-16-20-park-avenue-waitara-nsw/",
  "https://www.stonerealestate.com.au/property/8577655-25-chepana-street-lake-cathie-nsw/",
  "https://www.stonerealestate.com.au/property/8577703-33-24-26-watt-street-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8577752-12-circulo-drive-copacabana-nsw/",
  "https://www.stonerealestate.com.au/property/8577865-71a-queen-street-narellan-nsw/",
  "https://www.stonerealestate.com.au/property/8578021-5-tununda-place-eschol-park-nsw/",
  "https://www.stonerealestate.com.au/property/8578050-54-91-95-john-whiteway-drive-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8578051-32-kilpa-road-wyongah-nsw/",
  "https://www.stonerealestate.com.au/property/8578133-709-3-pretoria-parade-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8578316-5-ray-street-toukley-nsw/",
  "https://www.stonerealestate.com.au/property/8578350-19-lakeland-circuit-harrington-park-nsw/",
  "https://www.stonerealestate.com.au/property/8578352-7-66-ernest-street-crows-nest-nsw/",
  "https://www.stonerealestate.com.au/property/8578359-2-13-willow-place-port-macquarie-nsw/",
  "https://www.stonerealestate.com.au/property/8578376-3-packard-close-killarney-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8578402-28-5-15-lamond-drive-turramurra-nsw/",
  "https://www.stonerealestate.com.au/property/8578407-91-villiers-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/8578461-28-koorabel-avenue-copacabana-nsw/",
  "https://www.stonerealestate.com.au/property/8578513-22-triller-street-aberglasslyn-nsw/",
  "https://www.stonerealestate.com.au/property/8578517-6-8-factory-street-north-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8578572-18-edden-street-bellbird-nsw/",
  "https://www.stonerealestate.com.au/property/8578697-82-tozer-street-west-kempsey-nsw/",
  "https://www.stonerealestate.com.au/property/8578731-7-yuruga-place-lindfield-nsw/",
  "https://www.stonerealestate.com.au/property/8578775-1802-1-boys-avenue-blacktown-nsw/",
  "https://www.stonerealestate.com.au/property/8578814-17b-88-98-marsden-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8579119-30-oswald-street-wilton-nsw/",
  "https://www.stonerealestate.com.au/property/8579258-5305-1a-morton-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8579262-21-23-reilly-street-liverpool-nsw/",
  "https://www.stonerealestate.com.au/property/8579416-813-19-partridge-avenue-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8579426-30a-oswald-street-wilton-nsw/",
  "https://www.stonerealestate.com.au/property/8579469-61-cultivation-road-austral-nsw/",
  "https://www.stonerealestate.com.au/property/8579516-9-jefferis-avenue-cobbitty-nsw/",
  "https://www.stonerealestate.com.au/property/8579653-32-19-21-central-coast-highway-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8579661-7-lytton-place-campbelltown-nsw/",
  "https://www.stonerealestate.com.au/property/8579783-57-astoria-road-wollert-vic/",
  "https://www.stonerealestate.com.au/property/8580033-46-oneill-street-brighton-le-sands-nsw/",
  "https://www.stonerealestate.com.au/property/8580034-5-curlew-crescent-eagleby-qld/",
  "https://www.stonerealestate.com.au/property/8580073-406-10-fielder-street-west-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8580107-35-colonial-street-campbelltown-nsw/",
  "https://www.stonerealestate.com.au/property/8580108-3-83-saywell-road-macquarie-fields-nsw/",
  "https://www.stonerealestate.com.au/property/8580241-12-wilfrid-street-macquarie-fields-nsw/",
  "https://www.stonerealestate.com.au/property/8580271-6-8-12-bourke-street-waterford-west-qld/",
  "https://www.stonerealestate.com.au/property/8580272-7-10-monash-road-loganlea-qld/",
  "https://www.stonerealestate.com.au/property/8580278-6a-peter-place-bligh-park-nsw/",
  "https://www.stonerealestate.com.au/property/8580361-36a-kaleula-crescent-kiama-nsw/",
  "https://www.stonerealestate.com.au/property/8580426-8-pitt-street-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/8580453-68-pollock-avenue-wyong-nsw/",
  "https://www.stonerealestate.com.au/property/8580537-27-13-19-bryant-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/8580562-2-4-virginia-street-north-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8580601-46-norman-street-mangerton-nsw/",
  "https://www.stonerealestate.com.au/property/8580637-1-125-lakin-street-bateau-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8580918-82-bloomfield-street-south-kempsey-nsw/",
  "https://www.stonerealestate.com.au/property/8580953-50b-blue-crane-close-tumbi-umbi-nsw/",
  "https://www.stonerealestate.com.au/property/8581076-3-brodie-circle-baulkham-hills-nsw/",
  "https://www.stonerealestate.com.au/property/8581178-18-altitude-street-north-richmond-nsw/",
  "https://www.stonerealestate.com.au/property/8581444-5-olive-grove-lenah-valley-tas/",
  "https://www.stonerealestate.com.au/property/8581636-15-adrian-close-bateau-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8581983-302-41a-manchester-drive-schofields-nsw/",
  "https://www.stonerealestate.com.au/property/8582162-26-second-avenue-north-warrawong-nsw/",
  "https://www.stonerealestate.com.au/property/8582169-3507-24-the-lanes-boulevard-mermaid-waters-qld/",
  "https://www.stonerealestate.com.au/property/8582300-92-whiting-street-labrador-qld/",
  "https://www.stonerealestate.com.au/property/8582476-9-46-48-keira-street-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8582493-2-74-robsons-road-keiraville-nsw/",
  "https://www.stonerealestate.com.au/property/8582511-4-14-amess-place-belconnen-act/",
  "https://www.stonerealestate.com.au/property/8582512-28-salisbury-street-casey-act/",
  "https://www.stonerealestate.com.au/property/8582520-42-durack-street-downer-act/",
  "https://www.stonerealestate.com.au/property/8582542-18-2-6-noel-street-north-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8582543-4-20-grey-street-keiraville-nsw/",
  "https://www.stonerealestate.com.au/property/8582548-42-59-67-corrimal-street-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8582717-2-35-waterford-street-kellyville-ridge-nsw/",
  "https://www.stonerealestate.com.au/property/8582754-606-102-swain-street-gungahlin-act/",
  "https://www.stonerealestate.com.au/property/8582821-4-clementine-place-mardi-nsw/",
  "https://www.stonerealestate.com.au/property/8582887-28a-reycroft-avenue-quakers-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8582903-52-gladys-street-rydalmere-nsw/",
  "https://www.stonerealestate.com.au/property/8583077-132-caves-beach-road-caves-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8583100-2-25-27-villiers-street-rockdale-nsw/",
  "https://www.stonerealestate.com.au/property/8583102-67-wonga-road-yowie-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8583103-8-67-warrangarree-drive-woronora-heights-nsw/",
  "https://www.stonerealestate.com.au/property/8583144-28-debono-drive-miners-rest-vic/",
  "https://www.stonerealestate.com.au/property/8583167-32-frances-street-northmead-nsw/",
  "https://www.stonerealestate.com.au/property/8583238-26-21-ijong-street-braddon-act/",
  "https://www.stonerealestate.com.au/property/8583239-27-1-beissel-street-belconnen-act/",
  "https://www.stonerealestate.com.au/property/8583255-26a-bunbury-road-macquarie-fields-nsw/",
  "https://www.stonerealestate.com.au/property/8583311-24a-pillar-street-bellbird-nsw/",
  "https://www.stonerealestate.com.au/property/8583406-959-the-entrance-road-forresters-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8583408-8-ivy-avenue-chain-valley-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8583409-1-15-oaks-avenue-dee-why-nsw/",
  "https://www.stonerealestate.com.au/property/8583410-8-dahlia-avenue-hamlyn-terrace-nsw/",
  "https://www.stonerealestate.com.au/property/8584155-3-87-caldarra-avenue-engadine-nsw/",
  "https://www.stonerealestate.com.au/property/8584159-5-38-morton-street-wollstonecraft-nsw/",
  "https://www.stonerealestate.com.au/property/8584243-24-43-railway-parade-engadine-nsw/",
  "https://www.stonerealestate.com.au/property/8584329-20-ferguson-street-cessnock-nsw/",
  "https://www.stonerealestate.com.au/property/8584336-1604-330-church-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8584349-1-27a-old-gympie-road-kallangur-qld/",
  "https://www.stonerealestate.com.au/property/8584402-8-33-yarrawonga-street-ngunnawal-act/",
  "https://www.stonerealestate.com.au/property/8584435-6-20-victoria-road-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8584437-17-73-75-flora-street-kirrawee-nsw/",
  "https://www.stonerealestate.com.au/property/8584452-11-127-duffield-road-kallangur-qld/",
  "https://www.stonerealestate.com.au/property/8584571-116a-river-road-greenwich-nsw/",
  "https://www.stonerealestate.com.au/property/8584624-12-dwon-street-wilton-nsw/",
  "https://www.stonerealestate.com.au/property/8584711-52-neptune-street-umina-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8584713-814-120-eastern-valley-way-belconnen-act/",
  "https://www.stonerealestate.com.au/property/8584714-12a-dwon-street-wilton-nsw/",
  "https://www.stonerealestate.com.au/property/8584724-2-12-rangers-road-cremorne-nsw/",
  "https://www.stonerealestate.com.au/property/8584725-level-3-3d-4-distillery-drive-pyrmont-nsw/",
  "https://www.stonerealestate.com.au/property/8584777-13-highcliff-crescent-north-lakes-qld/",
  "https://www.stonerealestate.com.au/property/8584785-113-1-sergeants-lane-st-leonards-nsw/",
  "https://www.stonerealestate.com.au/property/8584786-8c-3-jersey-road-artarmon-nsw/",
  "https://www.stonerealestate.com.au/property/8584787-5-43-quarry-road-dundas-nsw/",
  "https://www.stonerealestate.com.au/property/8584820-48-lord-street-roseville-nsw/",
  "https://www.stonerealestate.com.au/property/8584841-2-338-scarborough-road-scarborough-qld/",
  "https://www.stonerealestate.com.au/property/8584845-2-29-austin-street-fairlight-nsw/",
  "https://www.stonerealestate.com.au/property/8584901-45-florence-street-towradgi-nsw/",
  "https://www.stonerealestate.com.au/property/8584902-23-derribong-drive-cordeaux-heights-nsw/",
  "https://www.stonerealestate.com.au/property/8584909-5-bullecourt-avenue-engadine-nsw/",
  "https://www.stonerealestate.com.au/property/8584915-84-22-peel-street-kirribilli-nsw/",
  "https://www.stonerealestate.com.au/property/8584962-6-brissendon-close-tarrawanna-nsw/",
  "https://www.stonerealestate.com.au/property/8584963-278-maitland-road-cessnock-nsw/",
  "https://www.stonerealestate.com.au/property/8584970-25-moorlan-ave-killarney-vale-nsw/",
  "https://www.stonerealestate.com.au/property/8585019-52-54-murray-park-road-figtree-nsw/",
  "https://www.stonerealestate.com.au/property/8585020-41-butterfactory-drive-calderwood-nsw/",
  "https://www.stonerealestate.com.au/property/8585022-1-21-arrow-avenue-figtree-nsw/",
  "https://www.stonerealestate.com.au/property/8585031-1-26-king-street-umina-beach-nsw/",
  "https://www.stonerealestate.com.au/property/8585102-24b-breynia-street-figtree-nsw/",
  "https://www.stonerealestate.com.au/property/8585103-202-47-51-crown-street-wollongong-nsw/",
  "https://www.stonerealestate.com.au/property/8585104-23-meadow-street-tarrawanna-nsw/",
  "https://www.stonerealestate.com.au/property/8585199-4-devlin-street-gungahlin-act/",
  "https://www.stonerealestate.com.au/property/8585270-23-congewai-street-kearsley-nsw/",
  "https://www.stonerealestate.com.au/property/8585279-32-3809-pacific-highway-tanah-merah-qld/",
  "https://www.stonerealestate.com.au/property/8585335-34-james-street-south-windsor-nsw/",
  "https://www.stonerealestate.com.au/property/8585434-4-abercorn-street-bexley-nsw/",
  "https://www.stonerealestate.com.au/property/8585487-36-main-street-smithtown-nsw/",
  "https://www.stonerealestate.com.au/property/8585546-5-38-40-harold-street-north-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8585589-14-55-57-hassall-street-westmead-nsw/",
  "https://www.stonerealestate.com.au/property/8585646-22-sturt-street-swansea-nsw/",
  "https://www.stonerealestate.com.au/property/8585649-50-38-orara-street-waitara-nsw/",
  "https://www.stonerealestate.com.au/property/8585779-26b-connel-drive-heddon-greta-nsw/",
  "https://www.stonerealestate.com.au/property/8585781-9-12-linda-street-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8585802-208-19-parramatta-road-homebush-nsw/",
  "https://www.stonerealestate.com.au/property/8585858-5-quintero-close-heddon-greta-nsw/",
  "https://www.stonerealestate.com.au/property/8585927-27-berkeley-street-speers-point-nsw/",
  "https://www.stonerealestate.com.au/property/8585930-28-fuller-avenue-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8585937-20-guernsey-avenue-minto-nsw/",
  "https://www.stonerealestate.com.au/property/8585987-6-605-king-street-newtown-nsw/",
  "https://www.stonerealestate.com.au/property/8586072-151-hampshire-boulevard-spring-farm-nsw/",
  "https://www.stonerealestate.com.au/property/8586143-4-14-victoria-parade-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8586144-5-bowral-crescent-tullimbar-nsw/",
  "https://www.stonerealestate.com.au/property/8586146-6-heidi-place-west-pennant-hills-nsw/",
  "https://www.stonerealestate.com.au/property/8586153-30-2-pound-road-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8586154-8-53-prospect-street-rosehill-nsw/",
  "https://www.stonerealestate.com.au/property/8586155-3-9-11-priddle-street-westmead-nsw/",
  "https://www.stonerealestate.com.au/property/8586159-14-john-hunter-grove-mount-annan-nsw/",
  "https://www.stonerealestate.com.au/property/8586223-46-pioneer-road-bellambi-nsw/",
  "https://www.stonerealestate.com.au/property/8586226-16-albert-street-north-lambton-nsw/",
  "https://www.stonerealestate.com.au/property/8586298-7-316-318-pennant-hills-road-carlingford-nsw/",
  "https://www.stonerealestate.com.au/property/8586305-4-garrard-street-spence-act/",
  "https://www.stonerealestate.com.au/property/8586355-1-luck-street-slacks-creek-qld/",
  "https://www.stonerealestate.com.au/property/8586465-24-keswick-avenue-castle-hill-nsw/",
  "https://www.stonerealestate.com.au/property/8586475-1401-41-45-waitara-avenue-waitara-nsw/",
  "https://www.stonerealestate.com.au/property/8586573-2-77-ruttleys-road-wyee-nsw/",
  "https://www.stonerealestate.com.au/property/8586574-13-caudalie-street-bellbird-nsw/",
  "https://www.stonerealestate.com.au/property/8586659-7-11-stuart-street-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8586660-11-46-48-solander-street-monterey-nsw/",
  "https://www.stonerealestate.com.au/property/8586662-411-pacific-highway-asquith-nsw/",
  "https://www.stonerealestate.com.au/property/8586668-5-174-gertrude-street-gosford-nsw/",
  "https://www.stonerealestate.com.au/property/8586677-18-11-17-watson-street-neutral-bay-nsw/",
  "https://www.stonerealestate.com.au/property/8586709-3-28-36-nursery-street-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8586888-204-32-34-ferntree-place-epping-nsw/",
  "https://www.stonerealestate.com.au/property/8586912-19-15-17-parc-guell-drive-campbelltown-nsw/",
  "https://www.stonerealestate.com.au/property/8586917-12-morrison-close-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/8586923-2-5-bruce-street-ashfield-nsw/",
  "https://www.stonerealestate.com.au/property/8587005-737-princes-highway-blakehurst-nsw/",
  "https://www.stonerealestate.com.au/property/8587027-15-46-48-bridge-road-hornsby-nsw/",
  "https://www.stonerealestate.com.au/property/8587029-5a-waterside-close-point-clare-nsw/",
  "https://www.stonerealestate.com.au/property/8587031-1914-3-grazier-lane-belconnen-act/",
  "https://www.stonerealestate.com.au/property/8587033-3-18-boronia-street-wollstonecraft-nsw/",
  "https://www.stonerealestate.com.au/property/8587037-6-12-star-street-sandy-bay-tas/",
  "https://www.stonerealestate.com.au/property/8587039-4-18-torrens-avenue-the-entrance-nsw/",
  "https://www.stonerealestate.com.au/property/8587104-717-22-central-avenue-manly-nsw/",
  "https://www.stonerealestate.com.au/property/8587114-3-william-cox-drive-richmond-nsw/",
  "https://www.stonerealestate.com.au/property/8587180-383-randalls-bay-road-eggs-and-bacon-bay-tas/",
  "https://www.stonerealestate.com.au/property/8587238-303-2d-appleroth-street-melrose-park-nsw/",
  "https://www.stonerealestate.com.au/property/8587280-9a-australia-road-barden-ridge-nsw/",
  "https://www.stonerealestate.com.au/property/8587359-10a-ferndale-street-chatswood-nsw/",
  "https://www.stonerealestate.com.au/property/8587360-4-22-hickey-street-cessnock-nsw/",
  "https://www.stonerealestate.com.au/property/8587452-level-g-5-16-hudson-street-lewisham-nsw/",
  "https://www.stonerealestate.com.au/property/8587588-3-11-powell-road-blackmans-bay-tas/",
  "https://www.stonerealestate.com.au/property/8587740-8-102-oconnell-street-north-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8587749-45-jacks-creek-road-whittlesea-vic/",
  "https://www.stonerealestate.com.au/property/8587784-8j-15-campbell-street-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8587825-48-stanford-street-kurri-kurri-nsw/",
  "https://www.stonerealestate.com.au/property/8587867-6-westerman-street-casey-act/",
  "https://www.stonerealestate.com.au/property/8587879-3-airdsley-lane-bradbury-nsw/",
  "https://www.stonerealestate.com.au/property/8587930-22-sunrise-drive-boambee-east-nsw/",
  "https://www.stonerealestate.com.au/property/8587971-500-bresnehans-road-little-swanport-tas/",
  "https://www.stonerealestate.com.au/property/8588022-77-mackays-road-coffs-harbour-nsw/",
  "https://www.stonerealestate.com.au/property/8588026-311-redwood-road-kingston-tas/",
  "https://www.stonerealestate.com.au/property/8588072-71-buller-street-north-parramatta-nsw/",
  "https://www.stonerealestate.com.au/property/8588074-56-178-184-ewing-road-logan-central-qld/",
  "https://www.stonerealestate.com.au/property/8588075-21-60-macarthy-road-marsden-qld/",
  "https://www.stonerealestate.com.au/property/8588088-11-roslyn-avenue-kingston-beach-tas/",
  "https://www.stonerealestate.com.au/property/8588238-35-mayfield-street-cessnock-nsw/",
  "https://www.stonerealestate.com.au/property/8588316-35-8-ken-tribe-street-coombs-act/",
  "https://www.stonerealestate.com.au/rent/",
  "https://www.stonerealestate.com.au/rent/commercial-lease/",
  "https://www.stonerealestate.com.au/rent/leasing-with-stone/",
  "https://www.stonerealestate.com.au/rent/property-alerts/",
  "https://www.stonerealestate.com.au/rent/request-appraisal/",
  "https://www.stonerealestate.com.au/rent/residential-lease/",
  "https://www.stonerealestate.com.au/sell/request-appraisal/",
  "https://www.stonerealestate.com.au/sell/selling-with-stone/",
  "https://www.stonerealestate.com.au/sold/",
  "https://www.stonerealestate.com.au/stone-tv-2/",
  "https://www.stonerealestate.com.au/strata/",
  "https://www.stonerealestate.com.au/terms-and-conditions/"
]

In [ ]:
# @title
agents = [
   "https://www.stonerealestate.com.au/stone-avalon-palm-beach/meet-team/scott-sinclair/",
"https://www.stonerealestate.com.au/stone-avalon-palm-beach/meet-team/simone-sinclair/",
"https://www.stonerealestate.com.au/stone-avoca-beach/meet-team/andrew-thomson/",
"https://www.stonerealestate.com.au/stone-avoca-beach/meet-team/christel-renton/",
"https://www.stonerealestate.com.au/stone-avoca-beach/meet-team/jason-carrozza/",
"https://www.stonerealestate.com.au/stone-ballarat/meet-team/dylan-archer_jeffrey/",
"https://www.stonerealestate.com.au/stone-ballarat/meet-team/fiona-hart/",
"https://www.stonerealestate.com.au/stone-ballarat/meet-team/jamie-gepp/",
"https://www.stonerealestate.com.au/stone-ballarat/meet-team/jarrod-kemp/",
"https://www.stonerealestate.com.au/stone-ballarat/meet-team/leigh-hutchinson/",
"https://www.stonerealestate.com.au/stone-ballarat/meet-team/rob-cunningham/",
"https://www.stonerealestate.com.au/stone-ballarat/meet-team/shane-kozaris/",
"https://www.stonerealestate.com.au/stone-ballarat/meet-team/shelleigh-chandler/",
"https://www.stonerealestate.com.au/stone-ballarat/meet-team/stu-brien/",
"https://www.stonerealestate.com.au/stone-ballarat/meet-team/tammy-edwards/",
"https://www.stonerealestate.com.au/stone-ballarat/meet-team/willow-eden/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/anne-farquharson/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/ashleigh-gray/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/chloe-mawass/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/chris-mayles/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/christie-irwin/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/cindy-xu/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/daniel-yao/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/danielle-bailey/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/danny-jing/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/eliza-zammit/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/ethan-wilson_williams/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/graham-black/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/iman-makari/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/james-ramsay/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/jane-booty/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/jeff-jianfu-luo/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/jessica-zhou/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/juliet-jiangmin-chen/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/karen-dangola/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/kevin-dearlove/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/kitty-lo/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/kylie-kelly/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/leo-li/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/marcus-dangola/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/mary-song/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/melanie-mun/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/mitchell-hubbard/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/penny-yates/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/rebecca-dearlove/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/renay-hall/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/sarah-hastings/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/sarah-langley/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/taj-mawass/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/tony-didd/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/vincent-goh/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/viviane-mylott/",
"https://www.stonerealestate.com.au/stone-beecroft/meet-team/wenjie-luo/",
"https://www.stonerealestate.com.au/stone-burleigh-heads-palm-beach/meet-team/alan-borges/",
"https://www.stonerealestate.com.au/stone-burleigh-heads-palm-beach/meet-team/amanda-blair/",
"https://www.stonerealestate.com.au/stone-burleigh-heads-palm-beach/meet-team/amy-andrews/",
"https://www.stonerealestate.com.au/stone-burleigh-heads-palm-beach/meet-team/brendan-andrews/",
"https://www.stonerealestate.com.au/stone-burleigh-heads-palm-beach/meet-team/diogo-braz/",
"https://www.stonerealestate.com.au/stone-burleigh-heads-palm-beach/meet-team/hollie-lewis/",
"https://www.stonerealestate.com.au/stone-burleigh-heads-palm-beach/meet-team/morgan-de-souza/",
"https://www.stonerealestate.com.au/stone-burleigh-heads-palm-beach/meet-team/rebecca-leo/",
"https://www.stonerealestate.com.au/stone-burleigh-heads-palm-beach/meet-team/scott-kelly/",
"https://www.stonerealestate.com.au/stone-burwood/meet-team/ben-chen/",
"https://www.stonerealestate.com.au/stone-burwood/meet-team/bow-yuning-jia/",
"https://www.stonerealestate.com.au/stone-burwood/meet-team/cathy-qin-chen/",
"https://www.stonerealestate.com.au/stone-burwood/meet-team/damon-zhiping-shen/",
"https://www.stonerealestate.com.au/stone-burwood/meet-team/jessica-yun/",
"https://www.stonerealestate.com.au/stone-burwood/meet-team/li-sammi-fan/",
"https://www.stonerealestate.com.au/stone-burwood/meet-team/penny-jiang/",
"https://www.stonerealestate.com.au/stone-burwood/meet-team/reception-burwood/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/antonia-shinn/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/belinda-taylor/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/caitlin-cameron/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/caleche-ventura/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/casey-king/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/charmaine-winch/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/chelsea-whitbread/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/dilini-ramanayaka/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/gary-luke/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/gemma-patterson/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/gisella-vohs/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/gladys-riley/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/kaitlin-hall/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/kelly-calderbank/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/kimberley-harley/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/kourtney-garrety/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/kristy-dangelo/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/leah-martin/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/lei-buddee/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/leonie-obrien/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/lynette-farrar/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/mariah-sassine/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/michelle-balabka/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/peter-laming/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/rebecca-david-nee-gray/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/russell-wyer/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/samantha-chea/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/sienna-kurdi/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/tamara-sadah/",
"https://www.stonerealestate.com.au/stone-campbelltown/meet-team/tanya-wickham/",
"https://www.stonerealestate.com.au/stone-carrara/meet-team/francine-gray/",
"https://www.stonerealestate.com.au/stone-carrara/meet-team/jason-starling/",
"https://www.stonerealestate.com.au/stone-carrara/meet-team/karen-ann/",
"https://www.stonerealestate.com.au/stone-carrara/meet-team/monique-koryzma/",
"https://www.stonerealestate.com.au/stone-carrara/meet-team/peter-wittman/",
"https://www.stonerealestate.com.au/stone-carrara/meet-team/scott-campbell/",
"https://www.stonerealestate.com.au/stone-carrara/meet-team/shaun-darcy/",
"https://www.stonerealestate.com.au/stone-coffs-harbour/meet-team/amber-klein/",
"https://www.stonerealestate.com.au/stone-coffs-harbour/meet-team/claudia-corfe/",
"https://www.stonerealestate.com.au/stone-coffs-harbour/meet-team/ellie-mckay/",
"https://www.stonerealestate.com.au/stone-coffs-harbour/meet-team/felicity-moore/",
"https://www.stonerealestate.com.au/stone-coffs-harbour/meet-team/lisa-covelli/",
"https://www.stonerealestate.com.au/stone-coffs-harbour/meet-team/luke-wenban/",
"https://www.stonerealestate.com.au/stone-coffs-harbour/meet-team/natasha-boyns/",
"https://www.stonerealestate.com.au/stone-coffs-harbour/meet-team/nicole-gill/",
"https://www.stonerealestate.com.au/stone-coffs-harbour/meet-team/savannah-mieni/",
"https://www.stonerealestate.com.au/stone-coffs-harbour/meet-team/scarlett-mackay/",
"https://www.stonerealestate.com.au/stone-coffs-harbour/meet-team/shane-brooks/",
"https://www.stonerealestate.com.au/stone-coffs-harbour/meet-team/yvette-ward/",
"https://www.stonerealestate.com.au/stone-commercial-inner-west/meet-team/benjamin-farhart/",
"https://www.stonerealestate.com.au/stone-commercial-inner-west/meet-team/daniel-costa/",
"https://www.stonerealestate.com.au/stone-commercial-inner-west/meet-team/joanne-choy/",
"https://www.stonerealestate.com.au/stone-commercial-inner-west/meet-team/john-bacic/",
"https://www.stonerealestate.com.au/stone-commercial-inner-west/meet-team/natalie-boland/",
"https://www.stonerealestate.com.au/stone-commercial-inner-west/meet-team/paul-di-mauro/",
"https://www.stonerealestate.com.au/stone-coomera/meet-team/michelle-evans/",
"https://www.stonerealestate.com.au/stone-coomera/meet-team/raquel-hannaway/",
"https://www.stonerealestate.com.au/stone-coomera/meet-team/rita_lily-evans/",
"https://www.stonerealestate.com.au/stone-coomera/meet-team/shane-evans/",
"https://www.stonerealestate.com.au/stone-coomera/meet-team/temo-colosimo/",
"https://www.stonerealestate.com.au/stone-coomera/meet-team/tiara-evans/",
"https://www.stonerealestate.com.au/stone-copacabana/meet-team/brent-pilkington/",
"https://www.stonerealestate.com.au/stone-copacabana/meet-team/caitlin-aristobil_adele/",
"https://www.stonerealestate.com.au/stone-copacabana/meet-team/peta-thomson/",
"https://www.stonerealestate.com.au/stone-corporate/meet-team/alexis-mumford/",
"https://www.stonerealestate.com.au/stone-corporate/meet-team/alison-prato/",
"https://www.stonerealestate.com.au/stone-corporate/meet-team/amanda-hunt/",
"https://www.stonerealestate.com.au/stone-corporate/meet-team/belinda-graham/",
"https://www.stonerealestate.com.au/stone-corporate/meet-team/ben-hamilton/",
"https://www.stonerealestate.com.au/stone-corporate/meet-team/brett-gay/",
"https://www.stonerealestate.com.au/stone-corporate/meet-team/gina-van-agten/",
"https://www.stonerealestate.com.au/stone-corporate/meet-team/graeme-hyde/",
"https://www.stonerealestate.com.au/stone-corporate/meet-team/jamie-bennett/",
"https://www.stonerealestate.com.au/stone-corporate/meet-team/luke-cameron/",
"https://www.stonerealestate.com.au/stone-corporate/meet-team/michael-conolly/",
"https://www.stonerealestate.com.au/stone-corporate/meet-team/peter-malouf/",
"https://www.stonerealestate.com.au/stone-corporate/meet-team/peter-mumford/",
"https://www.stonerealestate.com.au/stone-corporate/meet-team/victoria-pryor/",
"https://www.stonerealestate.com.au/stone-corporate/meet-team/zander-snape/",
"https://www.stonerealestate.com.au/stone-crows-nest/meet-team/casey-kowalczyk/",
"https://www.stonerealestate.com.au/stone-crows-nest/meet-team/charles-baynie/",
"https://www.stonerealestate.com.au/stone-crows-nest/meet-team/charlie-ayles/",
"https://www.stonerealestate.com.au/stone-crows-nest/meet-team/daniel-settineri/",
"https://www.stonerealestate.com.au/stone-crows-nest/meet-team/david-hill/",
"https://www.stonerealestate.com.au/stone-crows-nest/meet-team/debora-tecchio/",
"https://www.stonerealestate.com.au/stone-crows-nest/meet-team/gabrielle-easy/",
"https://www.stonerealestate.com.au/stone-crows-nest/meet-team/heather-winstanley/",
"https://www.stonerealestate.com.au/stone-crows-nest/meet-team/kaye-lagasca/",
"https://www.stonerealestate.com.au/stone-crows-nest/meet-team/ruby-johnson/",
"https://www.stonerealestate.com.au/stone-crows-nest/meet-team/sam-hill/",
"https://www.stonerealestate.com.au/stone-crows-nest/meet-team/sharon-curran/",
"https://www.stonerealestate.com.au/stone-dee-why/meet-team/catherine-wilson/",
"https://www.stonerealestate.com.au/stone-dee-why/meet-team/olivia-mumford/",
"https://www.stonerealestate.com.au/stone-dee-why/meet-team/tim-mumford/",
"https://www.stonerealestate.com.au/stone-dee-why/meet-team/wally-mumford/",
"https://www.stonerealestate.com.au/stone-drummoyne-concord/meet-team/buster-redman/",
"https://www.stonerealestate.com.au/stone-drummoyne-concord/meet-team/irma-rinaudo/",
"https://www.stonerealestate.com.au/stone-drummoyne-concord/meet-team/jennifer-howard/",
"https://www.stonerealestate.com.au/stone-drummoyne-concord/meet-team/jessica-loh-campos/",
"https://www.stonerealestate.com.au/stone-drummoyne-concord/meet-team/joseph-tropiano/",
"https://www.stonerealestate.com.au/stone-drummoyne-concord/meet-team/molly-wedd/",
"https://www.stonerealestate.com.au/stone-drummoyne-concord/meet-team/phillip-massa/",
"https://www.stonerealestate.com.au/stone-drummoyne-concord/meet-team/simone-tropiano/",
"https://www.stonerealestate.com.au/stone-earlwood/meet-team/caitlin-grayson/",
"https://www.stonerealestate.com.au/stone-earlwood/meet-team/david-porcellato/",
"https://www.stonerealestate.com.au/stone-earlwood/meet-team/dean-haustead/",
"https://www.stonerealestate.com.au/stone-earlwood/meet-team/evan-skagias/",
"https://www.stonerealestate.com.au/stone-earlwood/meet-team/george-karafotias/",
"https://www.stonerealestate.com.au/stone-earlwood/meet-team/harry-karafotias/",
"https://www.stonerealestate.com.au/stone-earlwood/meet-team/louise-veitch/",
"https://www.stonerealestate.com.au/stone-earlwood/meet-team/michael-alidenes/",
"https://www.stonerealestate.com.au/stone-earlwood/meet-team/stavroula-skagias/",
"https://www.stonerealestate.com.au/stone-earlwood/meet-team/victoria-lica-reynolds/",
"https://www.stonerealestate.com.au/stone-engadine/meet-team/amanda-scott/",
"https://www.stonerealestate.com.au/stone-engadine/meet-team/georgia-rollason/",
"https://www.stonerealestate.com.au/stone-engadine/meet-team/gerard-foote/",
"https://www.stonerealestate.com.au/stone-engadine/meet-team/lisa-christey/",
"https://www.stonerealestate.com.au/stone-engadine/meet-team/melanie-gardner/",
"https://www.stonerealestate.com.au/stone-engadine/meet-team/niki-hogan/",
"https://www.stonerealestate.com.au/stone-engadine/meet-team/saskia-roberts/",
"https://www.stonerealestate.com.au/stone-engadine/meet-team/tarni-payne/",
"https://www.stonerealestate.com.au/stone-engadine/meet-team/wade-gilmore/",
"https://www.stonerealestate.com.au/stone-engadine/meet-team/will-soulos/",
"https://www.stonerealestate.com.au/stone-ermington/meet-team/alex-bartels/",
"https://www.stonerealestate.com.au/stone-ermington/meet-team/bobby-yu/",
"https://www.stonerealestate.com.au/stone-ermington/meet-team/brandon-chai/",
"https://www.stonerealestate.com.au/stone-ermington/meet-team/charlie-wu/",
"https://www.stonerealestate.com.au/stone-ermington/meet-team/katherine-arias/",
"https://www.stonerealestate.com.au/stone-ermington/meet-team/melanie-carlson/",
"https://www.stonerealestate.com.au/stone-ermington/meet-team/stephen-breene/",
"https://www.stonerealestate.com.au/stone-ettalong/meet-team/charlie-killorn/",
"https://www.stonerealestate.com.au/stone-ettalong/meet-team/christine-glass/",
"https://www.stonerealestate.com.au/stone-ettalong/meet-team/ella-maskill/",
"https://www.stonerealestate.com.au/stone-ettalong/meet-team/helen-hughes/",
"https://www.stonerealestate.com.au/stone-ettalong/meet-team/lee-summers/",
"https://www.stonerealestate.com.au/stone-ettalong/meet-team/lian-ogrady/",
"https://www.stonerealestate.com.au/stone-ettalong/meet-team/margaret-marshall/",
"https://www.stonerealestate.com.au/stone-ettalong/meet-team/marnie-oneill/",
"https://www.stonerealestate.com.au/stone-ettalong/meet-team/tamara-lee/",
"https://www.stonerealestate.com.au/stone-ettalong/meet-team/tania-conroy/",
"https://www.stonerealestate.com.au/stone-ettalong/meet-team/zen-frediani/",
"https://www.stonerealestate.com.au/stone-forest/meet-team/alicia-sudol/",
"https://www.stonerealestate.com.au/stone-forest/meet-team/andrew-toohey/",
"https://www.stonerealestate.com.au/stone-forest/meet-team/david-hayden/",
"https://www.stonerealestate.com.au/stone-forest/meet-team/ruby-gibson/",
"https://www.stonerealestate.com.au/stone-forest/meet-team/sharon-stewart/",
"https://www.stonerealestate.com.au/stone-glenorie/meet-team/amanda-coles/",
"https://www.stonerealestate.com.au/stone-glenorie/meet-team/keith-hilliard/",
"https://www.stonerealestate.com.au/stone-glenorie/meet-team/natalie-thornton/",
"https://www.stonerealestate.com.au/stone-glenorie/meet-team/rhys-coles/",
"https://www.stonerealestate.com.au/stone-glenorie/meet-team/sally-thomson/",
"https://www.stonerealestate.com.au/stone-gosford/meet-team/adrian-knight/",
"https://www.stonerealestate.com.au/stone-gosford/meet-team/bridie-burgess_jones/",
"https://www.stonerealestate.com.au/stone-gosford/meet-team/emma-slocombe/",
"https://www.stonerealestate.com.au/stone-gosford/meet-team/jake-smith/",
"https://www.stonerealestate.com.au/stone-gosford/meet-team/kayla-knight/",
"https://www.stonerealestate.com.au/stone-gosford/meet-team/siena-pipiciello/",
"https://www.stonerealestate.com.au/stone-gosford/meet-team/tahlia-poulton_turner/",
"https://www.stonerealestate.com.au/stone-gosford/meet-team/taylah-miller/",
"https://www.stonerealestate.com.au/stone-greenwich/meet-team/bianca-karabulut/",
"https://www.stonerealestate.com.au/stone-greenwich/meet-team/chris-keane/",
"https://www.stonerealestate.com.au/stone-greenwich/meet-team/desiree-hough/",
"https://www.stonerealestate.com.au/stone-greenwich/meet-team/emma-barwood/",
"https://www.stonerealestate.com.au/stone-greenwich/meet-team/erin-renford/",
"https://www.stonerealestate.com.au/stone-greenwich/meet-team/georgina-williams/",
"https://www.stonerealestate.com.au/stone-greenwich/meet-team/jaydn-frintzilas/",
"https://www.stonerealestate.com.au/stone-greenwich/meet-team/lisa-brink/",
"https://www.stonerealestate.com.au/stone-greenwich/meet-team/roy-mccullagh/",
"https://www.stonerealestate.com.au/stone-greenwich/meet-team/tamara-charlton/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/alex-ford/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/alex-ogbonna/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/amanda-tehle/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/courtney-saddi/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/georgia-ryman/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/hazel-lim/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/hugh-rainger/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/hugo-mendez/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/isabella-ferry/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/izabell-cowan/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/jess-campbell/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/jess-doolan/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/jess-smith/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/kerry-robinson/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/lagan-puri/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/luke-bone/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/maxwell-jensen/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/riley-borthwick/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/riya-detwani/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/sam-dyne/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/shaye-davies/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/sienna-hoang/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/steve-boden/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/tahlia-dal-broi/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/teri-lim/",
"https://www.stonerealestate.com.au/stone-gungahlin/meet-team/xahli-stanford/",
"https://www.stonerealestate.com.au/stone-hawkesbury/meet-team/adam-buchert/",
"https://www.stonerealestate.com.au/stone-hawkesbury/meet-team/amanda-farrugia/",
"https://www.stonerealestate.com.au/stone-hawkesbury/meet-team/bernie-tate/",
"https://www.stonerealestate.com.au/stone-hawkesbury/meet-team/briana-mcdonald/",
"https://www.stonerealestate.com.au/stone-hawkesbury/meet-team/bridgett-creswell/",
"https://www.stonerealestate.com.au/stone-hawkesbury/meet-team/cathy-walton/",
"https://www.stonerealestate.com.au/stone-hawkesbury/meet-team/faithe-wheeler/",
"https://www.stonerealestate.com.au/stone-hawkesbury/meet-team/kayla-sisinni/",
"https://www.stonerealestate.com.au/stone-hawkesbury/meet-team/lee-boon/",
"https://www.stonerealestate.com.au/stone-hawkesbury/meet-team/michael-ohare/",
"https://www.stonerealestate.com.au/stone-hawkesbury/meet-team/nicky-browne/",
"https://www.stonerealestate.com.au/stone-hawkesbury/meet-team/soraiah-jones/",
"https://www.stonerealestate.com.au/stone-hawkesbury/meet-team/sydney-dobbin_hale/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/adam-noakes/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/angel-qinghua-li/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/april-angel/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/dean-finch/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/dianne-di-finch/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/fiona-noakes/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/georgie-miller/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/geraldine-xiao_bin-wang/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/jack-mcdonald/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/jaime-garrick/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/maia-fuchs/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/mark-seymour/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/mathew-ferraro/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/nicola-williams/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/penelope-lovell/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/ron-che_wei-shih/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/sophie-noakes/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/stephen-helies/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/steve-noakes/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/tina-brown/",
"https://www.stonerealestate.com.au/stone-hornsby/meet-team/zachary-oshaughnessy/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/adrienne-mcinnes/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/brayden-bean/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/brendan-dewley/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/cruz-tanchevski/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/debra-ford/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/donney-ristevski/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/faz-assir/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/jacqueline-tanko/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/jasmine-tammekand/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/jorja-pauling/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/kate-shaw/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/katrina-vitnell/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/kerrie-reilly/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/kim-holford/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/matthew-clarkson/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/michael-hurley/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/nicole-tarasenko/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/renae-redmond/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/renee-bean_wyper/",
"https://www.stonerealestate.com.au/stone-hunter-valley/meet-team/shelby-mason/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/aaron-dean/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/abbey-tilden/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/angela-bolton/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/ayla-paull/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/barbara-cooper/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/benjamin-sayers/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/charli-carriage/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/chloe-gee/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/darnell-haselau/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/david-scioscia/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/deanne-bannerman/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/emily-bolton/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/emma-fitzgibbon/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/greg-crumpton/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/hannah-portelli/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/harriet-saunders/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/hayley-clark/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/helena-crumpton/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/jake-mckinnon/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/jamie-pereira/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/jane-pearson/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/jessica-boyde/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/kara-shipp/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/kylie-davis/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/lauren-mellor/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/matthew-kasbarian/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/neil-webster/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/olivia-morlando/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/paul-piacentin/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/raffaele-di-candia/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/rebecca-carlson/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/rebecca-saunders/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/rebecca-veljanoska/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/sarah-ward/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/simon-medina/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/tahlia-venegas/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/vanessa-mcclelland/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/victoria-kolevski/",
"https://www.stonerealestate.com.au/stone-illawarra/meet-team/yenson-mui/",
"https://www.stonerealestate.com.au/stone-kempsey/meet-team/amanda-mcdonnell/",
"https://www.stonerealestate.com.au/stone-kempsey/meet-team/amie-jensen/",
"https://www.stonerealestate.com.au/stone-kempsey/meet-team/braithan-lewis/",
"https://www.stonerealestate.com.au/stone-kempsey/meet-team/carlos-peters/",
"https://www.stonerealestate.com.au/stone-kempsey/meet-team/hayley-cooper/",
"https://www.stonerealestate.com.au/stone-kempsey/meet-team/josh-wicks/",
"https://www.stonerealestate.com.au/stone-kempsey/meet-team/rebecca-omullane/",
"https://www.stonerealestate.com.au/stone-kempsey/meet-team/tracey-smith/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/alexandra-diktakis/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/allyson-andrean/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/ashley-booth/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/chelsey-girvan/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/corrine-wall/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/courtney-matheson/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/david-tanchevski/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/debbie-cheers/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/jesse-rowbottom/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/kathie-kidd/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/leanne-proctor/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/lisa-howard/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/roxanne-breeden/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/team-tanchevski/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/tegan-chaffer/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/tony-rich/",
"https://www.stonerealestate.com.au/stone-lake-macquarie/meet-team/trish-angus/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/adette-cao/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/alex-liapis/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/andrew-braid/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/anna-golubic/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/chloe-huang/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/claudia-conroy/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/daniel-patterson/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/darinka-vasilovska/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/ivy-wang/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/leo-zanetic/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/marco-arata/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/max-ma/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/niamh-barber/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/rosanna-morley/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/ryan-woo/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/steven-kourdis/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/sylvia-van-wyk/",
"https://www.stonerealestate.com.au/stone-lindfield/meet-team/vivie-li/",
"https://www.stonerealestate.com.au/stone-liverpool/meet-team/angela-mcinerney/",
"https://www.stonerealestate.com.au/stone-liverpool/meet-team/ashton-del-principe/",
"https://www.stonerealestate.com.au/stone-liverpool/meet-team/daniela-simic/",
"https://www.stonerealestate.com.au/stone-liverpool/meet-team/danielle-reynolds/",
"https://www.stonerealestate.com.au/stone-liverpool/meet-team/eevet-arabo/",
"https://www.stonerealestate.com.au/stone-liverpool/meet-team/frank-bartolone/",
"https://www.stonerealestate.com.au/stone-liverpool/meet-team/glen-craigie/",
"https://www.stonerealestate.com.au/stone-liverpool/meet-team/jessica-baron/",
"https://www.stonerealestate.com.au/stone-liverpool/meet-team/kaylee-lewis/",
"https://www.stonerealestate.com.au/stone-liverpool/meet-team/michael-rudan/",
"https://www.stonerealestate.com.au/stone-liverpool/meet-team/sophia-nguyen/",
"https://www.stonerealestate.com.au/stone-logan-west/meet-team/amber-gunn/",
"https://www.stonerealestate.com.au/stone-logan-west/meet-team/bridgette-griffiths/",
"https://www.stonerealestate.com.au/stone-logan-west/meet-team/claire-franklin/",
"https://www.stonerealestate.com.au/stone-logan-west/meet-team/darren-gunn/",
"https://www.stonerealestate.com.au/stone-logan-west/meet-team/leanne-gunn/",
"https://www.stonerealestate.com.au/stone-logan-west/meet-team/lee-johnstone/",
"https://www.stonerealestate.com.au/stone-logan-west/meet-team/leeanne-dench/",
"https://www.stonerealestate.com.au/stone-logan-west/meet-team/roslyn-watts/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/billy-bakos/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/cameron-rice/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/cassandra-magro/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/chantel-doherty/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/claire-nicholls/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/david-neate/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/denise-chetkett/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/eva-point/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/evyon-hinge/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/ibrahim-sulvic/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/janeva-meichtry/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/jason-good/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/jason-savage/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/lucy-meng/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/phoebe-cruickshank/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/polita-lennon/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/rebecca-suddrey/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/renee-savage/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/samantha-kuchler/",
"https://www.stonerealestate.com.au/stone-logan/meet-team/shail-wadhwa/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/adam-tassone/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/amy-goodwin/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/andrew-diadyk/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/andrew-pink/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/anthony-grima/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/bennett-parkinson/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/brittany-jarvis/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/cameron-mitchell/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/chris-philp/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/christina-buttigieg/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/danielle-evans/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/danielle-pollard/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/irene-mortakis/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/kylie-mason/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/mark-bricknell/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/melissa-avery/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/michelle-dowd/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/michelle-oconnell/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/omeka-tautaiolefue-saufoi/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/rachael-bartolone/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/rachel-micallef/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/rebecca-philp/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/renee-avery/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/sally-newton/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/sarah-ellaway/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/sarah-gigg/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/sarah-hales/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/shane-mackaway/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/susan-hayes/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/tony-soldo/",
"https://www.stonerealestate.com.au/stone-macarthur/meet-team/troy-nicholson/",
"https://www.stonerealestate.com.au/stone-manly/meet-team/adam-moore/",
"https://www.stonerealestate.com.au/stone-manly/meet-team/candice-cattell/",
"https://www.stonerealestate.com.au/stone-manly/meet-team/franki-watkinson/",
"https://www.stonerealestate.com.au/stone-manly/meet-team/janice-collings/",
"https://www.stonerealestate.com.au/stone-manly/meet-team/jenna-mcinnes/",
"https://www.stonerealestate.com.au/stone-manly/meet-team/mark-skeens/",
"https://www.stonerealestate.com.au/stone-manly/meet-team/michael-egan/",
"https://www.stonerealestate.com.au/stone-manly/meet-team/morgan-allen/",
"https://www.stonerealestate.com.au/stone-manly/meet-team/nathan-mckeown/",
"https://www.stonerealestate.com.au/stone-manly/meet-team/reuben-garrick/",
"https://www.stonerealestate.com.au/stone-manly/meet-team/ryan-wilson/",
"https://www.stonerealestate.com.au/stone-manly/meet-team/sam-bursill/",
"https://www.stonerealestate.com.au/stone-manly/meet-team/zahlia-frize/",
"https://www.stonerealestate.com.au/stone-maroubra/meet-team/dino-caruso/",
"https://www.stonerealestate.com.au/stone-maroubra/meet-team/eddy-wilson/",
"https://www.stonerealestate.com.au/stone-maroubra/meet-team/falentina-young/",
"https://www.stonerealestate.com.au/stone-maroubra/meet-team/finni-trin/",
"https://www.stonerealestate.com.au/stone-maroubra/meet-team/ivan-gunawan/",
"https://www.stonerealestate.com.au/stone-maroubra/meet-team/jason-yang/",
"https://www.stonerealestate.com.au/stone-maroubra/meet-team/karlina-widjaja/",
"https://www.stonerealestate.com.au/stone-maroubra/meet-team/mel-tjong/",
"https://www.stonerealestate.com.au/stone-maroubra/meet-team/ricky-regito/",
"https://www.stonerealestate.com.au/stone-maroubra/meet-team/ronald-jusuf/",
"https://www.stonerealestate.com.au/stone-mosman/meet-team/alanna-tohme/",
"https://www.stonerealestate.com.au/stone-mosman/meet-team/bruce-nhuang/",
"https://www.stonerealestate.com.au/stone-mosman/meet-team/christine-oconnell/",
"https://www.stonerealestate.com.au/stone-mosman/meet-team/eleni-kay/",
"https://www.stonerealestate.com.au/stone-mosman/meet-team/ellie-mcclenaghan/",
"https://www.stonerealestate.com.au/stone-mosman/meet-team/gemma-reynolds/",
"https://www.stonerealestate.com.au/stone-mosman/meet-team/marize-bellomo/",
"https://www.stonerealestate.com.au/stone-mosman/meet-team/patrick-short/",
"https://www.stonerealestate.com.au/stone-neutral-bay/meet-team/andrew-thomas/",
"https://www.stonerealestate.com.au/stone-neutral-bay/meet-team/eddie-tremolada/",
"https://www.stonerealestate.com.au/stone-neutral-bay/meet-team/ivan-resnekov/",
"https://www.stonerealestate.com.au/stone-neutral-bay/meet-team/jacqui-stanley/",
"https://www.stonerealestate.com.au/stone-neutral-bay/meet-team/joe-akle/",
"https://www.stonerealestate.com.au/stone-neutral-bay/meet-team/kate-baker/",
"https://www.stonerealestate.com.au/stone-neutral-bay/meet-team/louise-lipovac/",
"https://www.stonerealestate.com.au/stone-neutral-bay/meet-team/monica-lipovac/",
"https://www.stonerealestate.com.au/stone-neutral-bay/meet-team/rebecca-cunningham/",
"https://www.stonerealestate.com.au/stone-neutral-bay/meet-team/suzi-zadel/",
"https://www.stonerealestate.com.au/stone-newcastle/meet-team/ashleigh-lawson/",
"https://www.stonerealestate.com.au/stone-newcastle/meet-team/brittnee-robinson/",
"https://www.stonerealestate.com.au/stone-newcastle/meet-team/brooke-degotardi/",
"https://www.stonerealestate.com.au/stone-newcastle/meet-team/emily-banks/",
"https://www.stonerealestate.com.au/stone-newcastle/meet-team/jessica-patterson/",
"https://www.stonerealestate.com.au/stone-newcastle/meet-team/justin-redpath/",
"https://www.stonerealestate.com.au/stone-newcastle/meet-team/mikayla-deans/",
"https://www.stonerealestate.com.au/stone-newcastle/meet-team/sean-redpath/",
"https://www.stonerealestate.com.au/stone-newcastle/meet-team/sienna-mason/",
"https://www.stonerealestate.com.au/stone-newcastle/meet-team/todd-mason/",
"https://www.stonerealestate.com.au/stone-newtown/meet-team/adrian-vassallo/",
"https://www.stonerealestate.com.au/stone-newtown/meet-team/brendan-vitnell/",
"https://www.stonerealestate.com.au/stone-newtown/meet-team/madeleine-forbes/",
"https://www.stonerealestate.com.au/stone-newtown/meet-team/michael-sanchez/",
"https://www.stonerealestate.com.au/stone-newtown/meet-team/paul-alam/",
"https://www.stonerealestate.com.au/stone-north-lakes/meet-team/cameron-bruce/",
"https://www.stonerealestate.com.au/stone-north-lakes/meet-team/david-bruce/",
"https://www.stonerealestate.com.au/stone-north-lakes/meet-team/dee-doyle/",
"https://www.stonerealestate.com.au/stone-north-lakes/meet-team/jarod-faulkner/",
"https://www.stonerealestate.com.au/stone-north-lakes/meet-team/paris-griffin/",
"https://www.stonerealestate.com.au/stone-north-lakes/meet-team/team-doyle/",
"https://www.stonerealestate.com.au/stone-north-lakes/meet-team/teresa-bruce/",
"https://www.stonerealestate.com.au/stone-north-ryde/meet-team/aiden-mostofi/",
"https://www.stonerealestate.com.au/stone-north-ryde/meet-team/james-sarzano/",
"https://www.stonerealestate.com.au/stone-north-ryde/meet-team/jason-segon/",
"https://www.stonerealestate.com.au/stone-north-ryde/meet-team/stone-north-ryde-property-management/",
"https://www.stonerealestate.com.au/stone-northmead/meet-team/christine-howe/",
"https://www.stonerealestate.com.au/stone-northmead/meet-team/john-partridge/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/adam-denina/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/alicia-doherty/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/ashton-beukers/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/bella-losurdo/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/brittany-beukers/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/cassie-zhang/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/chloe-hurley/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/chris-kypreos/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/isabelle-theseira/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/jack-ferguson/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/jenny-denina/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/joseph-maatouk/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/luke-russo/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/luna-bing/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/mark-hurley/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/natalie-abreu/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/natasha-chand/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/patrick-boondok/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/paul-grech/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/scott-partridge/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/shivani-mandri/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/tava-wekwete/",
"https://www.stonerealestate.com.au/stone-parramatta/meet-team/zoe-pilotto/",
"https://www.stonerealestate.com.au/stone-pittwater/meet-team/mary-rowley/",
"https://www.stonerealestate.com.au/stone-port-macquarie/meet-team/ally-downing/",
"https://www.stonerealestate.com.au/stone-port-macquarie/meet-team/david-geary/",
"https://www.stonerealestate.com.au/stone-port-macquarie/meet-team/luke-hadfield/",
"https://www.stonerealestate.com.au/stone-port-macquarie/meet-team/meagan-ellison/",
"https://www.stonerealestate.com.au/stone-port-macquarie/meet-team/nathan-owen/",
"https://www.stonerealestate.com.au/stone-port-macquarie/meet-team/samet-portakaldali/",
"https://www.stonerealestate.com.au/stone-port-macquarie/meet-team/taylar-veech/",
"https://www.stonerealestate.com.au/stone-projects/meet-team/cameron-chung/",
"https://www.stonerealestate.com.au/stone-projects/meet-team/carl-heath/",
"https://www.stonerealestate.com.au/stone-projects/meet-team/jacqueline-dodd/",
"https://www.stonerealestate.com.au/stone-projects/meet-team/marshall-cobb/",
"https://www.stonerealestate.com.au/stone-projects/meet-team/md-kam-kamruzzaman/",
"https://www.stonerealestate.com.au/stone-projects/meet-team/nathan-shaw/",
"https://www.stonerealestate.com.au/stone-projects/meet-team/noah-reinecke/",
"https://www.stonerealestate.com.au/stone-projects/meet-team/pamela-cook/",
"https://www.stonerealestate.com.au/stone-projects/meet-team/paris-le-boursicot/",
"https://www.stonerealestate.com.au/stone-projects/meet-team/rafae-khan/",
"https://www.stonerealestate.com.au/stone-projects/meet-team/terry-zhu/",
"https://www.stonerealestate.com.au/stone-projects/meet-team/warren-plumb/",
"https://www.stonerealestate.com.au/stone-real-estate-roseville-willoughby/meet-team/alex-andonian/",
"https://www.stonerealestate.com.au/stone-real-estate-roseville-willoughby/meet-team/bridget-sutherland/",
"https://www.stonerealestate.com.au/stone-real-estate-roseville-willoughby/meet-team/evonne-chen/",
"https://www.stonerealestate.com.au/stone-real-estate-roseville-willoughby/meet-team/hugh-macfarlan/",
"https://www.stonerealestate.com.au/stone-real-estate-roseville-willoughby/meet-team/jill-henry/",
"https://www.stonerealestate.com.au/stone-real-estate-roseville-willoughby/meet-team/jimmy-psaltis/",
"https://www.stonerealestate.com.au/stone-real-estate-roseville-willoughby/meet-team/lisa-larcher/",
"https://www.stonerealestate.com.au/stone-real-estate-roseville-willoughby/meet-team/natalie-ramsay/",
"https://www.stonerealestate.com.au/stone-real-estate-roseville-willoughby/meet-team/nicola-quinn_smith/",
"https://www.stonerealestate.com.au/stone-real-estate-roseville-willoughby/meet-team/sahil-kotian/",
"https://www.stonerealestate.com.au/stone-real-estate-roseville-willoughby/meet-team/tiffany-yuan/",
"https://www.stonerealestate.com.au/stone-rockdale/meet-team/alexandra-amorginos/",
"https://www.stonerealestate.com.au/stone-rockdale/meet-team/dan-nguyen/",
"https://www.stonerealestate.com.au/stone-rockdale/meet-team/diana-gosarevska/",
"https://www.stonerealestate.com.au/stone-rockdale/meet-team/ej-joseph/",
"https://www.stonerealestate.com.au/stone-rockdale/meet-team/george-kambouroglou/",
"https://www.stonerealestate.com.au/stone-rockdale/meet-team/lilyan-muna/",
"https://www.stonerealestate.com.au/stone-rockdale/meet-team/omar-haouchar/",
"https://www.stonerealestate.com.au/stone-rockdale/meet-team/sam-abbas/",
"https://www.stonerealestate.com.au/stone-rockdale/meet-team/terry-gareb/",
"https://www.stonerealestate.com.au/stone-rockdale/meet-team/tijana-krstova/",
"https://www.stonerealestate.com.au/stone-sans-souci/meet-team/aleks-stankovic/",
"https://www.stonerealestate.com.au/stone-sans-souci/meet-team/alexi-kounnas/",
"https://www.stonerealestate.com.au/stone-sans-souci/meet-team/caitlin-morrison/",
"https://www.stonerealestate.com.au/stone-sans-souci/meet-team/chloe-cooper/",
"https://www.stonerealestate.com.au/stone-sans-souci/meet-team/daniel-gao/",
"https://www.stonerealestate.com.au/stone-sans-souci/meet-team/gerry-filas/",
"https://www.stonerealestate.com.au/stone-sans-souci/meet-team/liam-roukis/",
"https://www.stonerealestate.com.au/stone-sans-souci/meet-team/michaela-gorkovski/",
"https://www.stonerealestate.com.au/stone-sans-souci/meet-team/mrinaal-duggal/",
"https://www.stonerealestate.com.au/stone-sans-souci/meet-team/nikola-tabakovic/",
"https://www.stonerealestate.com.au/stone-sans-souci/meet-team/ray-fadel/",
"https://www.stonerealestate.com.au/stone-sans-souci/meet-team/rory-kalos/",
"https://www.stonerealestate.com.au/stone-sans-souci/meet-team/shaun-ramani/",
"https://www.stonerealestate.com.au/stone-sans-souci/meet-team/zac-comninos/",
"https://www.stonerealestate.com.au/stone-schofields-quakers-hill/meet-team/andre-wazen/",
"https://www.stonerealestate.com.au/stone-schofields-quakers-hill/meet-team/gagan-bindra/",
"https://www.stonerealestate.com.au/stone-schofields-quakers-hill/meet-team/karan-thapar/",
"https://www.stonerealestate.com.au/stone-schofields-quakers-hill/meet-team/krupa-modhia/",
"https://www.stonerealestate.com.au/stone-schofields-quakers-hill/meet-team/nenad-kovacevic/",
"https://www.stonerealestate.com.au/stone-schofields-quakers-hill/meet-team/rental-team/",
"https://www.stonerealestate.com.au/stone-schofields-quakers-hill/meet-team/vrajesh-soni/",
"https://www.stonerealestate.com.au/stone-seaforth/meet-team/alexis-lex-nobilo/",
"https://www.stonerealestate.com.au/stone-seaforth/meet-team/anthony-zakos/",
"https://www.stonerealestate.com.au/stone-seaforth/meet-team/cathy-shields/",
"https://www.stonerealestate.com.au/stone-seaforth/meet-team/chris-elliott/",
"https://www.stonerealestate.com.au/stone-seaforth/meet-team/chris-lewis/",
"https://www.stonerealestate.com.au/stone-seaforth/meet-team/claire-kinnaird/",
"https://www.stonerealestate.com.au/stone-seaforth/meet-team/harry-cass/",
"https://www.stonerealestate.com.au/stone-seaforth/meet-team/kirsty-hunt/",
"https://www.stonerealestate.com.au/stone-seaforth/meet-team/lauren-sudol/",
"https://www.stonerealestate.com.au/stone-seaforth/meet-team/louise-ashdown/",
"https://www.stonerealestate.com.au/stone-seaforth/meet-team/maria-cassarino/",
"https://www.stonerealestate.com.au/stone-seaforth/meet-team/marilyn-farquhar/",
"https://www.stonerealestate.com.au/stone-seaforth/meet-team/nicole-wesley/",
"https://www.stonerealestate.com.au/stone-seaforth/meet-team/sarah-cass/",
"https://www.stonerealestate.com.au/stone-seaforth/meet-team/todd-baker/",
"https://www.stonerealestate.com.au/stone-south-hurstville/meet-team/alannah-rawlinson/",
"https://www.stonerealestate.com.au/stone-south-hurstville/meet-team/jacob-sant/",
"https://www.stonerealestate.com.au/stone-south-hurstville/meet-team/jordan-damato/",
"https://www.stonerealestate.com.au/stone-southern-highlands/meet-team/alexander-kolovos/",
"https://www.stonerealestate.com.au/stone-southern-highlands/meet-team/george-kolovos/",
"https://www.stonerealestate.com.au/stone-southern-highlands/meet-team/telissa-clout/",
"https://www.stonerealestate.com.au/stone-southern-tasmania/meet-team/chris-johnston/",
"https://www.stonerealestate.com.au/stone-southern-tasmania/meet-team/cloe-smith/",
"https://www.stonerealestate.com.au/stone-southern-tasmania/meet-team/jane-bendall/",
"https://www.stonerealestate.com.au/stone-southern-tasmania/meet-team/kimberley-dodge/",
"https://www.stonerealestate.com.au/stone-southern-tasmania/meet-team/louise-wallace/",
"https://www.stonerealestate.com.au/stone-southern-tasmania/meet-team/maddison-ayres/",
"https://www.stonerealestate.com.au/stone-southern-tasmania/meet-team/sally-thomas/",
"https://www.stonerealestate.com.au/stone-southern-tasmania/meet-team/wendy-wade/",
"https://www.stonerealestate.com.au/stone-southport/meet-team/aydan-mullin/",
"https://www.stonerealestate.com.au/stone-southport/meet-team/colin-george/",
"https://www.stonerealestate.com.au/stone-southport/meet-team/lauren-mahon/",
"https://www.stonerealestate.com.au/stone-southport/meet-team/marcello-giardina/",
"https://www.stonerealestate.com.au/stone-southport/meet-team/mitchell-bean/",
"https://www.stonerealestate.com.au/stone-southport/meet-team/stefanie-mullin/",
"https://www.stonerealestate.com.au/stone-southport/meet-team/trinity-hollingsworth/",
"https://www.stonerealestate.com.au/stone-terrigal/meet-team/andrea-johnson/",
"https://www.stonerealestate.com.au/stone-terrigal/meet-team/david-lyle/",
"https://www.stonerealestate.com.au/stone-terrigal/meet-team/hans-belle/",
"https://www.stonerealestate.com.au/stone-terrigal/meet-team/michelle-cattle/",
"https://www.stonerealestate.com.au/stone-terrigal/meet-team/ray-moore/",
"https://www.stonerealestate.com.au/stone-terrigal/meet-team/samantha-washington/",
"https://www.stonerealestate.com.au/stone-terrigal/meet-team/sarah-gilchrist/",
"https://www.stonerealestate.com.au/stone-terrigal/meet-team/shane-marini/",
"https://www.stonerealestate.com.au/stone-toukley/meet-team/jasmine-heggs/",
"https://www.stonerealestate.com.au/stone-toukley/meet-team/louise-barwick/",
"https://www.stonerealestate.com.au/stone-toukley/meet-team/tegan-ashman/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/abdullah-choudhari/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/alicia-ortiz/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/alicia-thornton/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/bianca-ross/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/brian-milson/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/ethan-akintetik/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/jamara-kennedy/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/jameel-hasan/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/jessica-caira/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/joey-lewis/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/kayla-bennett/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/kristy-baker/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/luke-villata/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/lynne-cowell/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/mark-gibb/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/mat-cairns/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/montanna-slattery/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/paije-wiggins/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/reece-caira/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/renee-wright/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/samantha-seagg/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/taylah-hall/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/tim-bembridge/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/tony-trinder/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/vanessa-martin/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/volkan-akintetik/",
"https://www.stonerealestate.com.au/stone-tumbi-umbi/meet-team/whitney-follett/",
"https://www.stonerealestate.com.au/stone-tv-2/",
"https://www.stonerealestate.com.au/stone-wahroonga-turramurra/meet-team/andy-howden/",
"https://www.stonerealestate.com.au/stone-wahroonga-turramurra/meet-team/christopher-smith/",
"https://www.stonerealestate.com.au/stone-wahroonga-turramurra/meet-team/diane-morgan/",
"https://www.stonerealestate.com.au/stone-wahroonga-turramurra/meet-team/dominic-conley/",
"https://www.stonerealestate.com.au/stone-wahroonga-turramurra/meet-team/esrin-saridas/",
"https://www.stonerealestate.com.au/stone-wahroonga-turramurra/meet-team/harry-bacales/",
"https://www.stonerealestate.com.au/stone-wahroonga-turramurra/meet-team/james-perry/",
"https://www.stonerealestate.com.au/stone-wahroonga-turramurra/meet-team/jemma-radocaj/",
"https://www.stonerealestate.com.au/stone-wahroonga-turramurra/meet-team/jessica-west/",
"https://www.stonerealestate.com.au/stone-wahroonga-turramurra/meet-team/joey-hoover/",
"https://www.stonerealestate.com.au/stone-wahroonga-turramurra/meet-team/ray-talati/",
"https://www.stonerealestate.com.au/stone-wahroonga-turramurra/meet-team/susan-astridge/",
"https://www.stonerealestate.com.au/stone-warner/meet-team/john-ward/",
"https://www.stonerealestate.com.au/stone-warner/meet-team/sue-ward/",
"https://www.stonerealestate.com.au/stone-warner/meet-team/zachary-ward/",
"https://www.stonerealestate.com.au/stone-waterloo/meet-team/jacky-weiqiu-lian/",
"https://www.stonerealestate.com.au/stone-waterloo/meet-team/jun-wu/",
"https://www.stonerealestate.com.au/stone-waterloo/meet-team/lily-liew/",
"https://www.stonerealestate.com.au/stone-waterloo/meet-team/michael-higgins/",
"https://www.stonerealestate.com.au/stone-waterloo/meet-team/paul-liao/",
"https://www.stonerealestate.com.au/stone-waterloo/meet-team/sabrina-xiaodan-wang/",
"https://www.stonerealestate.com.au/stone-whittlesea/meet-team/angie-ermanis/",
"https://www.stonerealestate.com.au/stone-whittlesea/meet-team/archie-miranda/",
"https://www.stonerealestate.com.au/stone-whittlesea/meet-team/chloe-dwyer_fraser/",
"https://www.stonerealestate.com.au/stone-whittlesea/meet-team/dean-zammit/",
"https://www.stonerealestate.com.au/stone-whittlesea/meet-team/fadzi-mutusva/",
"https://www.stonerealestate.com.au/stone-whittlesea/meet-team/farai-katiyo/",
"https://www.stonerealestate.com.au/stone-whittlesea/meet-team/indy-davies/",
"https://www.stonerealestate.com.au/stone-whittlesea/meet-team/jenni-ter-haar/",
"https://www.stonerealestate.com.au/stone-whittlesea/meet-team/jessica-conte_singh/",
"https://www.stonerealestate.com.au/stone-whittlesea/meet-team/kristin-parker/",
"https://www.stonerealestate.com.au/stone-whittlesea/meet-team/sienna-saikaly/",
"https://www.stonerealestate.com.au/stone-whittlesea/meet-team/suzzi-jan/",
"https://www.stonerealestate.com.au/stone-wyong/meet-team/alyce-lewis/",
"https://www.stonerealestate.com.au/stone-wyong/meet-team/brooke-brown/",
"https://www.stonerealestate.com.au/stone-wyong/meet-team/jason-cozens/",
"https://www.stonerealestate.com.au/stone-wyong/meet-team/kaela-wall/",
"https://www.stonerealestate.com.au/stone-wyong/meet-team/kit-gillings/",
"https://www.stonerealestate.com.au/stone-wyong/meet-team/scott-wall/",
"https://www.stonerealestate.com.au/strata/"
]

In [ ]:
blogs = [
"https://www.stonerealestate.com.au/2022-real-estate-market-forecast/",
"https://www.stonerealestate.com.au/3-reasons-you-should-use-indoor-plants-to-bring-your-living-spaces-to-life/",
"https://www.stonerealestate.com.au/5-tips-how-to-style-every-room-in-your-home-this-christmas/",
"https://www.stonerealestate.com.au/a-5-step-survival-guide-for-hosting-christmas/",
"https://www.stonerealestate.com.au/a-three-step-guide-to-buying-your-first-investment-property/",
"https://www.stonerealestate.com.au/a-waterfront-for-under-a-million-absolutely/",
"https://www.stonerealestate.com.au/apartment-vs-house/",
"https://www.stonerealestate.com.au/attention-all-landlords/",
"https://www.stonerealestate.com.au/awards-for-excellence/",
"https://www.stonerealestate.com.au/be-like-a-ferrari/",
"https://www.stonerealestate.com.au/bean-craving/",
"https://www.stonerealestate.com.au/beauty-and-the-brick/",
"https://www.stonerealestate.com.au/bee-good/",
"https://www.stonerealestate.com.au/brave-hearts/",
"https://www.stonerealestate.com.au/breaking-records/",
"https://www.stonerealestate.com.au/buying-your-first-home-in-qld-what-you-need-to-know/",
"https://www.stonerealestate.com.au/buying-your-first-home-what-you-need-to-know/",
"https://www.stonerealestate.com.au/careers/",
"https://www.stonerealestate.com.au/cashing-in-its-a-sellers-market/",
"https://www.stonerealestate.com.au/clutter-chaos/",
"https://www.stonerealestate.com.au/cocktails-and-canapes/",
"https://www.stonerealestate.com.au/declutter-your-2020/",
"https://www.stonerealestate.com.au/do-you-have-to-stay-with-the-same-property-manager/",
"https://www.stonerealestate.com.au/eat-out-home/",
"https://www.stonerealestate.com.au/education-before-motivation/",
"https://www.stonerealestate.com.au/embrace-winter-the-best-outdoor-activities-in-australia/",
"https://www.stonerealestate.com.au/endless-love/",
"https://www.stonerealestate.com.au/family-in-a-flat-could-you-would-you/",
"https://www.stonerealestate.com.au/first-home-buyers-victoria-what-you-need-to-know/",
"https://www.stonerealestate.com.au/first-home-owner-grant/",
"https://www.stonerealestate.com.au/flips-and-tricks/",
"https://www.stonerealestate.com.au/four-factors-that-will-influence-the-property-market-in-2023/",
"https://www.stonerealestate.com.au/franchise/",
"https://www.stonerealestate.com.au/friday-favourites-best-of-the-new/",
"https://www.stonerealestate.com.au/fridays-favourite-fun-features/",
"https://www.stonerealestate.com.au/fridays-favourites-3-hot-properties-on-the-market/",
"https://www.stonerealestate.com.au/fridays-favourites-4-indoor-outdoor-dining-areas/",
"https://www.stonerealestate.com.au/fridays-favourites-4-waterside-wonders/",
"https://www.stonerealestate.com.au/fridays-favourites-5-beautiful-facades/",
"https://www.stonerealestate.com.au/fridays-favourites-5-dream-kitchens/",
"https://www.stonerealestate.com.au/fridays-favourites-5-dream-properties-to-rent/",
"https://www.stonerealestate.com.au/fridays-favourites-designer-inspired/",
"https://www.stonerealestate.com.au/fridays-favourites-our-top-5-acreages/",
"https://www.stonerealestate.com.au/fridays-favourites-our-top-5-outdoor-dining-areas/",
"https://www.stonerealestate.com.au/fridays-favourites-our-top-five-home-features/",
"https://www.stonerealestate.com.au/fridays-favourites-secret-spots/",
"https://www.stonerealestate.com.au/fridays-favourites-single-level-living/",
"https://www.stonerealestate.com.au/fridays-favourites-the-best-five-pools-this-week/",
"https://www.stonerealestate.com.au/fridays-favourites-what-are-you-looking-at/",
"https://www.stonerealestate.com.au/fridays-favourites/",
"https://www.stonerealestate.com.au/giving-back-to-our-communities/",
"https://www.stonerealestate.com.au/green-is-the-new-black/",
"https://www.stonerealestate.com.au/group-buying-providing-security-for-mum-dad-investors/",
"https://www.stonerealestate.com.au/happy-in-a-hamptons/",
"https://www.stonerealestate.com.au/healthy-winter-home/",
"https://www.stonerealestate.com.au/homeworkis-not-a-dirty-word/",
"https://www.stonerealestate.com.au/hot-house/",
"https://www.stonerealestate.com.au/hot-property/",
"https://www.stonerealestate.com.au/how-to-increase-the-value-of-your-property-with-photography/",
"https://www.stonerealestate.com.au/how-to-maximise-your-rental-income/",
"https://www.stonerealestate.com.au/in-just-5-years-chris-philp-soars-from-agent-to-business-owner-of-5-offices/",
"https://www.stonerealestate.com.au/inside-a-contemporary-crib-in-sans-souci/",
"https://www.stonerealestate.com.au/inside-a-luxury-residence-in-mosman/",
"https://www.stonerealestate.com.au/international-womens-day-angela-bolton/",
"https://www.stonerealestate.com.au/international-womens-day-annika-bongiornos-story/",
"https://www.stonerealestate.com.au/international-womens-day-deanne-bannerman/",
"https://www.stonerealestate.com.au/international-womens-day-jill-henrys-story/",
"https://www.stonerealestate.com.au/international-womens-day-maddie-thornton/",
"https://www.stonerealestate.com.au/international-womens-day-samantha-xerris-story/",
"https://www.stonerealestate.com.au/is-a-sea-or-tree-change-the-right-move-for-you/",
"https://www.stonerealestate.com.au/island-escapade/",
"https://www.stonerealestate.com.au/its-bath-time/",
"https://www.stonerealestate.com.au/its-showtime-top-tips-for-selling-your-home-in-spring/",
"https://www.stonerealestate.com.au/just-like-property-prices-building-costs-are-soaring/",
"https://www.stonerealestate.com.au/just-like-the-weather-the-property-market-is-different-in-each-state/",
"https://www.stonerealestate.com.au/keeping-your-plants-alive-through-winter/",
"https://www.stonerealestate.com.au/knock-knock-who-lives-there/",
"https://www.stonerealestate.com.au/land-grab/",
"https://www.stonerealestate.com.au/lets-take-it-outside/",
"https://www.stonerealestate.com.au/lets-talk-about-tech/",
"https://www.stonerealestate.com.au/leveraging-leading-artificial-intelligence-technology/",
"https://www.stonerealestate.com.au/love-at-first-sight/",
"https://www.stonerealestate.com.au/love-your-day-job/",
"https://www.stonerealestate.com.au/macarthur-region-nsw/",
"https://www.stonerealestate.com.au/making-working-from-home-work/",
"https://www.stonerealestate.com.au/making-your-move/",
"https://www.stonerealestate.com.au/manage/",
"https://www.stonerealestate.com.au/more-than-a-makeover/",
"https://www.stonerealestate.com.au/most-liveable-suburbs-in-sydney-is-your-suburb-on-the-list/",
"https://www.stonerealestate.com.au/movember-our-stone-mo-bros/",
"https://www.stonerealestate.com.au/navigating-the-new-rental-laws-in-new-south-wales/",
"https://www.stonerealestate.com.au/new-and-newsworthy-12-june-2020/",
"https://www.stonerealestate.com.au/new-and-newsworthy-2/",
"https://www.stonerealestate.com.au/new-and-newsworthy-28-august-2020/",
"https://www.stonerealestate.com.au/new-and-newsworthy-property-highlights-for-week-ending-11th-september/",
"https://www.stonerealestate.com.au/new-and-newsworthy-this-weeks-property-highlights/",
"https://www.stonerealestate.com.au/new-and-newsworthy/",
"https://www.stonerealestate.com.au/northern-beaches-nirvana/",
"https://www.stonerealestate.com.au/not-just-for-granny-the-potential-and-the-power-of-the-granny-flat/",
"https://www.stonerealestate.com.au/our-approach-to-management/",
"https://www.stonerealestate.com.au/our-approach-to-selling/",
"https://www.stonerealestate.com.au/our-guide-to-selling-off-market/",
"https://www.stonerealestate.com.au/our-people/join-us/",
"https://www.stonerealestate.com.au/our-people/our-agents/",
"https://www.stonerealestate.com.au/our-people/our-offices/",
"https://www.stonerealestate.com.au/our-stone-women-event/",
"https://www.stonerealestate.com.au/plus-plus-doubling-up/",
"https://www.stonerealestate.com.au/potentially-loaded/",
"https://www.stonerealestate.com.au/power-full/",
"https://www.stonerealestate.com.au/prepare-your-garden-for-summer-in-three-simple-steps/",
"https://www.stonerealestate.com.au/privacy-policy/",
"https://www.stonerealestate.com.au/projects/",
"https://www.stonerealestate.com.au/property-market-update/",
"https://www.stonerealestate.com.au/property-report/",
"https://www.stonerealestate.com.au/rate-my-agent-awards-2024/",
"https://www.stonerealestate.com.au/ratemyagent-agent-of-the-year-awards-2020/",
"https://www.stonerealestate.com.au/read-street/",
"https://www.stonerealestate.com.au/rent/",
"https://www.stonerealestate.com.au/rent/commercial-lease/",
"https://www.stonerealestate.com.au/rent/leasing-with-stone/",
"https://www.stonerealestate.com.au/rent/property-alerts/",
"https://www.stonerealestate.com.au/rent/request-appraisal/",
"https://www.stonerealestate.com.au/rent/residential-lease/",
"https://www.stonerealestate.com.au/rma-awards-2022/",
"https://www.stonerealestate.com.au/room-to-live-work-and-play/",
"https://www.stonerealestate.com.au/save-planet-earth/",
"https://www.stonerealestate.com.au/sell/request-appraisal/",
"https://www.stonerealestate.com.au/sell/selling-with-stone/",
"https://www.stonerealestate.com.au/selling-in-2021/",
"https://www.stonerealestate.com.au/selling-this-summer-heres-your-to-do-list/",
"https://www.stonerealestate.com.au/semi-precious/",
"https://www.stonerealestate.com.au/shaking-up-stamp-duty-in-nsw/",
"https://www.stonerealestate.com.au/six-reasons-why-you-need-a-property-manager/",
"https://www.stonerealestate.com.au/small-space-big-style/",
"https://www.stonerealestate.com.au/sold-the-story-of-two-extremely-quick-sales/",
"https://www.stonerealestate.com.au/sold/",
"https://www.stonerealestate.com.au/splish-splash/",
"https://www.stonerealestate.com.au/spring-market-snapshot-three-key-indicators/",
"https://www.stonerealestate.com.au/stone-opens-in-wahroonga/",
"https://www.stonerealestate.com.au/stone-opens-two-new-offices-in-south-western-sydney/",
"https://www.stonerealestate.com.au/stone-real-estate-adds-new-franchise-location-with-hawkesbury-office/",
"https://www.stonerealestate.com.au/stone-real-estate-continues-its-queensland-expansion-launching-stone-logan-west/",
"https://www.stonerealestate.com.au/stone-real-estate-continues-queensland-expansion-with-golden-beach-office/",
"https://www.stonerealestate.com.au/stone-real-estate-continues-rapid-sydney-expansion-with-greenwich-office/",
"https://www.stonerealestate.com.au/stone-real-estate-expands-central-coast-presence/",
"https://www.stonerealestate.com.au/stone-real-estate-expands-in-greater-melbourne-with-whittlesea-office/",
"https://www.stonerealestate.com.au/stone-real-estate-further-strengthens-its-corporate-team-with-the-valuable-addition-of-an-industry-icon/",
"https://www.stonerealestate.com.au/stone-real-estate-lands-in-tasmania/",
"https://www.stonerealestate.com.au/stone-real-estate-launches-glenorie-office/",
"https://www.stonerealestate.com.au/stone-real-estate-opens-central-sydney-office-in-waterloo/",
"https://www.stonerealestate.com.au/stone-real-estate-opens-new-franchise-in-warner-queensland/",
"https://www.stonerealestate.com.au/stone-real-estate-opens-new-office-in-carrara-gold-coast/",
"https://www.stonerealestate.com.au/stone-real-estate-opens-new-office-in-gymea/",
"https://www.stonerealestate.com.au/stone-real-estate-opens-office-in-north-lakes-kallangur-area-of-queensland/",
"https://www.stonerealestate.com.au/stone-real-estate-strengthens-gold-coast-presence-with-southport-office/",
"https://www.stonerealestate.com.au/stone-real-estate-welcomes-stone-kempsey/",
"https://www.stonerealestate.com.au/stone-real-estate-welcomes-stone-northmead/",
"https://www.stonerealestate.com.au/stone-real-estates-mid-north-coast-journey-begins-with-the-grand-opening-of-stone-coffs-harbour/",
"https://www.stonerealestate.com.au/stone-tv-2/",
"https://www.stonerealestate.com.au/strata/",
"https://www.stonerealestate.com.au/suburban-oasis/",
"https://www.stonerealestate.com.au/super-saturday/",
"https://www.stonerealestate.com.au/take-a-drive-be-tempted/",
"https://www.stonerealestate.com.au/teenager-in-the-house/",
"https://www.stonerealestate.com.au/terms-and-conditions/",
"https://www.stonerealestate.com.au/the-asian-century/",
"https://www.stonerealestate.com.au/the-beauty-of-the-one-bedder/",
"https://www.stonerealestate.com.au/the-benefits-of-a-property-appraisal/",
"https://www.stonerealestate.com.au/the-best-of-the-best-the-reb-top-100/",
"https://www.stonerealestate.com.au/the-best-of-the-new-this-week/",
"https://www.stonerealestate.com.au/the-deal-on-developing/",
"https://www.stonerealestate.com.au/the-deep-blue/",
"https://www.stonerealestate.com.au/the-door-is-open-how-to-make-an-entrance/",
"https://www.stonerealestate.com.au/the-great-cook-out/",
"https://www.stonerealestate.com.au/the-home-or-the-hood/",
"https://www.stonerealestate.com.au/the-importance-of-tenant-screening/",
"https://www.stonerealestate.com.au/the-perfect-australian-fireplace/",
"https://www.stonerealestate.com.au/the-pros-and-cons-of-selling-in-summer/",
"https://www.stonerealestate.com.au/the-pros-cons-of-rent-vesting/",
"https://www.stonerealestate.com.au/the-queenslander-a-design-ahead-of-its-time/",
"https://www.stonerealestate.com.au/the-re-change-making-the-move-to-regional/",
"https://www.stonerealestate.com.au/the-real-appeal-of-renting/",
"https://www.stonerealestate.com.au/the-shifting-status-of-the-freestanding-home/",
"https://www.stonerealestate.com.au/the-treat-of-tricking-halloween-in-your-hood/",
"https://www.stonerealestate.com.au/the-trick-to-treating-in-2020-halloween-covid-19-style/",
"https://www.stonerealestate.com.au/the-truth-about-holiday-homes/",
"https://www.stonerealestate.com.au/thinking-about-investing-in-property/",
"https://www.stonerealestate.com.au/thinking-of-selling/",
"https://www.stonerealestate.com.au/three-reasons-why-now-just-might-be-the-perfect-time-to-sell/",
"https://www.stonerealestate.com.au/top-5-home-interior-trends-for-2020/",
"https://www.stonerealestate.com.au/top-9-interior-design-trends-for-modern-homes-in-2023/",
"https://www.stonerealestate.com.au/try-before-you-buy-with-our-kitchen-styling-tool/",
"https://www.stonerealestate.com.au/want-to-know-the-most-searched-suburbs-in-nsw/",
"https://www.stonerealestate.com.au/welcome-stone-ballarat-buninyong/",
"https://www.stonerealestate.com.au/welcome-stone-bateau-bay/",
"https://www.stonerealestate.com.au/welcome-stone-burwood/",
"https://www.stonerealestate.com.au/welcome-stone-doreen/",
"https://www.stonerealestate.com.au/welcome-stone-drummoyne-concord/",
"https://www.stonerealestate.com.au/welcome-stone-ermington/",
"https://www.stonerealestate.com.au/welcome-stone-liverpool/",
"https://www.stonerealestate.com.au/welcome-stone-maroubra/",
"https://www.stonerealestate.com.au/welcome-stone-neutral-bay/",
"https://www.stonerealestate.com.au/welcome-stone-newcastle/",
"https://www.stonerealestate.com.au/welcome-stone-real-estate-port-macquarie/",
"https://www.stonerealestate.com.au/welcome-stone-south-hurstville/",
"https://www.stonerealestate.com.au/welcome-stone-swansea/",
"https://www.stonerealestate.com.au/what-buyers-want-five-features-hopeful-homeowners-are-seeking-out/",
"https://www.stonerealestate.com.au/what-buyers-want-the-top-searches-revealed/",
"https://www.stonerealestate.com.au/what-buyers-want/",
"https://www.stonerealestate.com.au/what-to-do-athome/",
"https://www.stonerealestate.com.au/whats-the-latest/",
"https://www.stonerealestate.com.au/whats-the-right-property-move-before-the-eofy/",
"https://www.stonerealestate.com.au/when-bigger-is-not-always-better/",
"https://www.stonerealestate.com.au/why-auctions-work-in-real-estate/",
"https://www.stonerealestate.com.au/why-breaking-up-with-your-tenant-is-hard-to-do/",
"https://www.stonerealestate.com.au/why-this-autumn-might-be-a-better-time-to-sell-than-ever/",
"https://www.stonerealestate.com.au/why-your-budget-for-fy23-will-be-the-most-important-budget-youll-need-to-prepare/",
"https://www.stonerealestate.com.au/winter-home-staging-tips-for-sellers/",
"https://www.stonerealestate.com.au/winter-wonderland/",
"https://www.stonerealestate.com.au/your-kitchen-rules/",
"https://www.stonerealestate.com.au/your-questions-our-answer/"
]

In [ ]:
#@title pages extractor
import os
import re
import requests
import trafilatura
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import shutil

# -----------------------------
# Helpers
# -----------------------------
def clean_filename(name):
    """Clean a string so it is safe as a filename."""
    name = re.sub(r'[^a-zA-Z0-9_-]', '_', name)
    name = re.sub(r'_+', '_', name)
    return name.strip("_")


def title_from_html(html):
    """Extract <title> and clean it."""
    soup = BeautifulSoup(html, "html.parser")
    title = soup.title.string if soup.title else "untitled_page"
    return clean_filename(title)


# -----------------------------
# Main Extraction Logic
# -----------------------------
output_dir = "stonerealestate-blogs"
zip_name = "stonerealestate-blogs"

os.makedirs(output_dir, exist_ok=True)

for url in blogs:
    try:
        print(f"[INFO] Fetching: {url}")
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        html_source = response.text

        # Extract title for both filenames
        title_clean = title_from_html(html_source)

        # -----------------------------
        # Save Raw HTML (title.html)
        # -----------------------------
        html_filename = f"{title_clean}.html"
        html_filepath = os.path.join(output_dir, html_filename)

        with open(html_filepath, "w", encoding="utf-8") as f:
            f.write(html_source)

        print(f"[HTML] Saved: {html_filepath}")

        # -----------------------------
        # Save Extracted Text (title.txt)
        # -----------------------------
        extracted_text = trafilatura.extract(html_source)
        if not extracted_text:
            print(f"[WARN] Could not extract main content from {url}")
            continue

        txt_filename = f"{title_clean}.txt"
        txt_filepath = os.path.join(output_dir, txt_filename)

        with open(txt_filepath, "w", encoding="utf-8") as f:
            f.write(f"{title_clean}\n\n")  # also add title inside the text
            f.write(extracted_text)

        print(f"[TEXT] Saved: {txt_filepath}")

    except requests.exceptions.RequestException as e:
        print(f"[ERROR] Failed to fetch {url} -> {e}")

# -----------------------------
# Zip the output directory
# -----------------------------
shutil.make_archive(zip_name, 'zip', output_dir)
print(f"\n[FINAL] All extracted files zipped into: {zip_name}.zip")

[INFO] Fetching: https://www.stonerealestate.com.au/2022-real-estate-market-forecast/
[HTML] Saved: stonerealestate-blogs/Stone_Real_Estate_2022_Property_Market_Forecast_Stone_Real_Estate.html
[TEXT] Saved: stonerealestate-blogs/Stone_Real_Estate_2022_Property_Market_Forecast_Stone_Real_Estate.txt
[INFO] Fetching: https://www.stonerealestate.com.au/3-reasons-you-should-use-indoor-plants-to-bring-your-living-spaces-to-life/
[HTML] Saved: stonerealestate-blogs/Stone_Real_Estate_3_reasons_you_should_use_indoor_plants_to_bring_your_living_spaces_to_life_Stone_Real_Estate.html
[TEXT] Saved: stonerealestate-blogs/Stone_Real_Estate_3_reasons_you_should_use_indoor_plants_to_bring_your_living_spaces_to_life_Stone_Real_Estate.txt
[INFO] Fetching: https://www.stonerealestate.com.au/5-tips-how-to-style-every-room-in-your-home-this-christmas/
[HTML] Saved: stonerealestate-blogs/Stone_Real_Estate_5_Tips_How_to_style_every_room_in_your_home_this_Christmas_Stone_Real_Estate.html
[TEXT] Saved: stonerea

In [5]:
import zipfile
import os

zip_file_path = '/content/stonerealestate-agents.zip'
extraction_path = '/content/stonerealestate_agents'

os.makedirs(extraction_path, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

print(f"Successfully extracted '{zip_file_path}' to '{extraction_path}'")

Successfully extracted '/content/stonerealestate-agents.zip' to '/content/stonerealestate_agents'


In [6]:
import os

def merge_text_files(folder_path, output_file):
    """
    Merge all .txt files in the specified folder into a single text file.
    Each file's content is separated by a header (the filename).
    """
    # Get all text files in folder
    text_files = [f for f in os.listdir(folder_path) if f.endswith(".txt")]

    if not text_files:
        print("No text files found in the folder.")
        return

    with open(output_file, "w", encoding="utf-8") as outfile:
        for filename in text_files:
            file_path = os.path.join(folder_path, filename)

            # Write header
            header = f"\n\n===== {filename} =====\n\n"
            outfile.write(header)

            # Write file content
            with open(file_path, "r", encoding="utf-8") as infile:
                content = infile.read()
                outfile.write(content)

    print(f"✅ Successfully merged {len(text_files)} files into '{output_file}'")

if __name__ == "__main__":
    # Example usage:
    folder = "/content/stonerealestate_agents"          # Folder containing text files
    output = "stonerealestate-agents.txt" # Output file name
    merge_text_files(folder, output)

✅ Successfully merged 739 files into 'stonerealestate-agents.txt'


In [8]:
import json
import os

input_directory = '/content/'
output_directory = '/content/extracted_properties_txt2'

os.makedirs(output_directory, exist_ok=True)

def format_property_to_text(property_data, file_prefix, index):
    # This function formats a single property dictionary
    text_content = f"=== PROPERTY {index + 1} ({file_prefix}) ===\n\n"
    for key, value in property_data.items():
        text_content += f"{str(key).replace('_', ' ').title()}: {value}\n"
    return text_content

json_files = [f for f in os.listdir(input_directory) if f.endswith('.json')]

for json_file in json_files:
    file_path = os.path.join(input_directory, json_file)
    file_prefix = os.path.splitext(json_file)[0]
    output_filename = os.path.join(output_directory, f"{file_prefix}.txt") # One text file per JSON file

    print(f"Processing: {json_file}")
    full_text_content = ""

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if isinstance(data, list):
            for i, property_data in enumerate(data):
                full_text_content += format_property_to_text(property_data, file_prefix, i) + "\n---\n\n"
        elif isinstance(data, dict):
            full_text_content += format_property_to_text(data, file_prefix, 0) + "\n---\n\n"
        else:
            print(f"  - Skipped: {json_file} (unsupported JSON format)")
            continue

        with open(output_filename, 'w', encoding='utf-8') as outfile:
            outfile.write(full_text_content)
        print(f"  - Saved all properties from {json_file} to {output_filename}")

    except json.JSONDecodeError as e:
        print(f"  - Error decoding JSON in {json_file}: {e}")
    except Exception as e:
        print(f"  - An unexpected error occurred while processing {json_file}: {e}")

print(f"\nAll processed files saved to: {output_directory}")

Processing: CommercialLease.json
  - Saved all properties from CommercialLease.json to /content/extracted_properties_txt2/CommercialLease.txt
Processing: ResidentialLease.json
  - Saved all properties from ResidentialLease.json to /content/extracted_properties_txt2/ResidentialLease.txt
Processing: residential_sales.json
  - Saved all properties from residential_sales.json to /content/extracted_properties_txt2/residential_sales.txt
Processing: off_market_sales.json
  - Saved all properties from off_market_sales.json to /content/extracted_properties_txt2/off_market_sales.txt
Processing: lease.json
  - Saved all properties from lease.json to /content/extracted_properties_txt2/lease.txt
Processing: commercial_sale.json
  - Saved all properties from commercial_sale.json to /content/extracted_properties_txt2/commercial_sale.txt
Processing: sold-out.json
  - Saved all properties from sold-out.json to /content/extracted_properties_txt2/sold-out.txt
Processing: acreage.json
  - Saved all proper

# Task
Compress the `/content/extracted_properties_txt2` folder into a zip archive named `extracted_properties_txt2.zip` and confirm that the folder has been compressed and is ready for download.

## Compress Folder

### Subtask:
Compress the `/content/extracted_properties_txt2` folder into a zip archive named `extracted_properties_txt2.zip`.


**Reasoning**:
To compress the specified folder into a zip archive, I need to use the `shutil.make_archive` function, as instructed.



In [9]:
import shutil

shutil.make_archive('extracted_properties_txt2', 'zip', '/content/extracted_properties_txt2')
print("Successfully created 'extracted_properties_txt2.zip' in /content/")

Successfully created 'extracted_properties_txt2.zip' in /content/


## Final Task

### Subtask:
Confirm that the folder has been compressed and is ready for download.


## Summary:

### Q&A
The folder has been successfully compressed and is ready for download.

### Data Analysis Key Findings
*   The `/content/extracted_properties_txt2` folder was successfully compressed into a zip archive.
*   The resulting zip archive is named `extracted_properties_txt2.zip`.
*   The `extracted_properties_txt2.zip` file was created in the `/content/` directory.

### Insights or Next Steps
*   The `extracted_properties_txt2.zip` file can now be downloaded from the `/content/` directory.
